# 陳時中、川普、柯文哲市長候選人大PK

Penalty Kick or Player Kill for Taipei city mayor candidates--Chen, Huang, and Jiang


What is share of voice? 品牌聲量

        Share of voice (SOV) is a measure of the market your brand owns compared to your competitors. It acts as a gauge for your brand visibility and how much you dominate the conversation in your industry. The more market share you have, the greater popularity and authority you likely have among users and prospective customers.

什麼是SOV？

        SOV又稱Share of Vocie，是衡量一切有關品牌聲量的指標，在數位時代，SOV包含品牌在特定關鍵字、搜尋引擎以及社群聲量的占比，是衡量多少消費者認知到您品牌的重要指標。
        
[來源](https://digitalpr.tw/%E7%A4%BE%E7%BE%A4%E5%8F%A3%E7%A2%91/sov%E7%A4%BE%E7%BE%A4%E8%81%B2%E9%87%8F%E5%81%A5%E6%AA%A2/)


# Put them together

In [143]:
import pandas as pd
from datetime import datetime, timedelta
import re

# Load Data 
df = pd.read_csv('./news_dataset_preprocessed_for_django.csv',sep='|')
# Step 0: Filter news articles using the following function
# Searching keywords from "content" column
def filter_df_via_content(df, query_keywords, cond, cate, weeks):

    # end date: the date of the latest record of news
    end_date = df.date.max()
    
    # start date
    start_date_delta = (datetime.strptime(end_date, '%Y-%m-%d').date() - timedelta(weeks=weeks)).strftime('%Y-%m-%d')
    start_date_min = df.date.min()
    # set start_date as the larger one from the start_date_delta and start_date_min 開始時間選資料最早時間與周數:兩者較晚者
    start_date = max(start_date_delta,   start_date_min)


    # (1) proceed filtering: a duration of a period of time
    # 期間條件
    period_condition = (df.date >= start_date) & (df.date <= end_date) 
    
    # (2) proceed filtering: news category
    # 新聞類別條件
    if (cate == "全部"):
        condition = period_condition  # "全部"類別不必過濾新聞種類
    else:
        # 過濾category新聞類別條件
        condition = period_condition & (df.category == cate)

    # (3) proceed filtering: and or
    # and or 條件
    if (cond == 'and'):
        # query keywords condition使用者輸入關鍵字條件and
        condition = condition & df.content.apply(lambda text: all((qk in text) for qk in query_keywords)) #寫法:all()
    elif (cond == 'or'):
        # query keywords condition使用者輸入關鍵字條件
        condition = condition & df.content.apply(lambda text: any((qk in text) for qk in query_keywords)) #寫法:any()
    # condiction is a list of True or False boolean value
    df_query = df[condition]

    return df_query

# Step 1: Sentimental polarity score計算整體情緒分數(影響力)
# sentimental polarity score
def get_article_sentiment(df_query):
    # df_query = df[df['tokens'].str.contains(query_key)]
    sentiCount = {'pos': 0, 'neg': 0, 'obj': 0}
    sentiPercnt = {'pos': 0, 'neg': 0, 'obj': 0}
    numberOfArticle = len(df_query)
    for senti in df_query.sentiment:
        # 判斷文章的情緒極性
        if senti >= 0.75:
            sentiCount['pos'] += 1
        elif senti <= 0.4:
            sentiCount['neg'] += 1
        else:
            sentiCount['obj'] += 1
    for polar in sentiCount :
        sentiPercnt[polar]=round(sentiCount[polar]/numberOfArticle*100)

    return sentiCount, sentiPercnt

# **計算各類別多少篇文章提到該關鍵字
# **計算各類別出現關鍵字次數

news_categories = ['政治', '科技', '運動', '證卷', '產經', '娛樂', '生活', '國際', '社會', '文化', '兩岸', '全部']



def count_keyword(df_query, query_keywords):

    cate_occurrence = {}
    cate_freq = {}
    
    # 字典初始化
    for cate in news_categories:
        cate_occurrence[cate] = 0   # {'政治':0, '科技':0}
        cate_freq[cate] = 0

    for idx, row in df_query.iterrows():
        # count the number of articles各類別篇數統計
        cate_occurrence[row.category] += 1  #   {'政治':+1, '科技':0}
        cate_occurrence['全部'] += 1
        
        # count the keyword frequency各類別次數統計
        # 計算這一篇文章的content中重複含有多少個這些關鍵字(頻率)
        freq = sum([ len(re.findall(keyword, row.content, re.I)) for keyword in query_keywords]) 
        cate_freq[row.category] += freq # 在該新聞類別中累計頻率
        cate_freq['全部'] += freq  # 在"全部"類別中累計頻率

    total_articles = cate_occurrence['全部']  # len(df_query)
    total_frequency = cate_freq['全部']
    return cate_freq, cate_occurrence, total_articles, total_frequency

# 與上一週的程式碼有些不一樣，讓每個PK的對象之資料的start_date end_date都相同，折線圖才會完整
def get_keyword_occurrence_time_series(df_query):


    # end date: the date of the latest record of news
    end_date = df.date.max()
    
    # start date
    start_date_delta = (datetime.strptime(end_date, '%Y-%m-%d').date() - timedelta(weeks=weeks)).strftime('%Y-%m-%d')
    start_date_min = df.date.min()
    # set start_date as the larger one from the start_date_delta and start_date_min 開始時間選資料最早時間與周數:兩者較晚者
    start_date = max(start_date_delta,   start_date_min)
    

    # 設定時間欄位'date_index', 次數freq欄位，將每一篇的時間寫到'date_index'欄位，將freq設定為1 ==> 進行groupby之後就可以合併日期與加總freq次數
    query_freq = pd.DataFrame({'date_index':pd.to_datetime( df_query.date ),'freq':[1 for _ in range(len(df_query))]})

    # 開始時間、結束時間兩項必須也加入到query_freq，計算次數時才會有完整的時間軸，否則時間軸長度會因為新聞時間不同，導致時間軸不一致
    dt_start_date = datetime.strptime(start_date, '%Y-%m-%d')
    dt_end_date = datetime.strptime(end_date, '%Y-%m-%d')   

    query_freq = pd.concat([query_freq, pd.DataFrame({'date_index': [dt_start_date], 'freq': [0]})])
    query_freq = pd.concat([query_freq, pd.DataFrame({'date_index': [dt_end_date], 'freq': [0]})])
    # query_freq = query_freq.append({'date_index': dt_end_date, 'freq': 0}, ignore_index=True)

    # 透過groupby就可以合併日期與加總freq次數
    freq_data_group = query_freq.groupby(pd.Grouper(key='date_index', freq='D')).sum()

    # 'date_index'為index(索引)，將其變成欄位名稱，inplace=True表示原始檔案會被異動
    freq_data_group.reset_index(inplace=True)
    # freq_data_group = freq_data_group.reset_index() # 這樣也可以得到同樣結果

    # 只有頻率次數值y, 沒有時間變數x, 計算相關係數時會用到
    # freq_data = freq_data_group.freq.to_list()

    # 有時間變數x,y，用於畫趨勢線圖
    line_xy_data = [{'x':date.strftime('%Y-%m-%d'),'y':freq} for date, freq in zip(freq_data_group.date_index,freq_data_group.freq)]

    return line_xy_data

# Step 4: Process PK data
# 定義PK對象
# 通常會多個關鍵字代表該PK對象，['柯文哲','柯p','柯P']
list_pkNames = ['馬斯克', '川普', '柯文哲']
list_pkKeywordSet = [['馬斯克'], ['川普'], ['柯文哲']]

# list_pkKeywordSet = [['馬斯克'], ['川普'], ['柯文哲'], ['柯文哲','柯p','柯P']]


# 線條顏色
list_colors = ["rgba(0,128,0,0.2)",'rgba(0,255,255,0.2)','rgba(0,0,255,0.2)']
#list_colors = ['green', 'red', 'blue']

# 人頭圖案
list_photos = [
    "data:image/webp;base64,UklGRmAWAABXRUJQVlA4WAoAAAAMAAAA4QAAmgAAVlA4ICYMAACwQwCdASriAJsAPpFCnEulo6KxpBDa+jASCWdt/IkYYM03ONCiTs+VIMbuwWWKAH8i/xHowaEPq39mfgR6QH7h+xQxVpkA0o65DXBCpBN8pXOp7PNbdQmjzx31hzv3guNEZFeUzndlAnBxndff8JntJ27pGPEW8ZXTvJjbks44/zbRD8SKcg4rBvSwtz6HKhwVev8fMCWofd9mHIZKXowkPpgwbWTeylxMve5SbAJLfBs0V0n8qlx8wcTHO0aqIJxyv85WgNz+0rMvaGQtGdEYuM8Z5DVlgXI0kf6kqFQATXmmLmPgeXA78c+jIvaEjNkFxHYx/baDg5tPy+H5YF1+L5z5pRtfMj3qQ0sVPqGZGtrGUrWhTj9F1Yiz/daIL88LsU7QBXg+23TUoLSntvbGFfUIOAXF6hs032rgIEYxV7mYVwYiUGQbG3Pc+iv6y1xA8FXfua/lZUmEx509lrzF94D/bd8N8UtDyGvXl2/FdvRR9HyDHbNBTV1Z93SPOD4LuGCQuSGra+ev6LhwbA4I+kLOWVOUIX1NxXe6UJj8gwSS5XYtJJoQ71xhEByHrAcycc4uLvZf29bG98+nx0Qt8e4OC/JPCrxG7rT9+zzyDTJVDAlU9EK3XTNkazuyvOGABjJPkgHvhl839Fm46MgXieF2QnSsvhlRjIx6+z1+4SvXKfi7+aRuMKWZkZ3wuxx1MgbNJoTS4seSmDoPEu5+Nz/3FGxYAP7qvqiWV4wML+DqzerCtHwmAg/ZQVwEjzzlmqLhj3dKu940x87bQbLdeGnSZGymF8UO1c8IfVeH23LkXJkxoOUdsDhwYjIxJbNCS0C5BFC18j6BxJz2F1epAvZSLkLf2bTRQ4b6d1NlegcfKlh8Ads/wcMN61RhlDWCR4rkuiofN/aCfPhsKFNkFv40EV8l6Ep0X99vH12aAJcHWVEq14xsuxKspuUYP6etI15sX+GMPEgXbqVwyF3qscm9mD6gNFu6bphl01ZhGznLsI2Wkyrua2nW7QgG2ILeMwDeP+if3XefghP/7rFzULUM5x6q/0jLlv+ji4OCE/5ivn+volZ4M2YaLr+u2252i/hfFb/c6AB004JXpw+ClsxR80QrwW2M/r9Pmrdgw+NBXO0nLXY4zpIouOW8p2PfHhwkEgaRP2YXbFgBtRm32FzTM8Fl58k6fARUIuH+lCeGan2biKiiArxQObp+Pegwy+7EgKsb2THO156VkKAPiPKQjfA8PU9hnuxSTWl+zUXUED+46geasqG8asyfzPUCIZdSpiy+YrZfiBZ4Zu7kMg5ABetH4OwY7a6Et01JNRnjSlAqLUuWYBwWRAQID2SJRRDcmZJZVl4wSqcllZ19n6XQ3SIT4VD5CBaL7bly6wlafY41Soaugy8K3+dAZyjmuEV24+tcNxhmMwKaCNuX1dDIhSTqtEdO05iGFo9XrSFnlFdmXyB/EDRsdaiBcXgmlIzem5jWdTrcrlO8VuijmIHFEe92Wg+Mh53ieR4pPpzstK1ePLAwptg/3fDc1ZZUE5/3oDvsdp8hXqPcbZps9CfYvZ5Bnwszo/CVkzHhP/ItuzAFnimAp/f2OprVQW466oKpdX7z1PcgmL4r98nh/z7M18ll65g66GMwgT7cAHYp7D5fazcETG2yBHvftsc5do0KJQVLZGgdSsjFhaWXxiJFcbVWAjl1yEbIGng5OqeJ7KdRDpwQkNue8lxBumDr6xG4mzixLBuNbhFWv4iDH8rFcuDqSIEmNkaLb2jTFAvBaH4+MqKF3SxgqAOyNZHx4KhZd5KKUZkOy7PDQmw0qhROMz4nEFocqdA9GNButm4LZB2oqBMNt6AG87rUaFLhwINHmfYbeCthEWAg++Y5fgTZzItgis6FBTY5poQxyVe57+TY+Bya67dmz5sQgO9ERbja0WvWUOTcxgr/bhqw1510NnKEaW3yrfic+lq0wTdN6xyqq0Aqy55LGdXn54uZ56iSSPLRemqxNP6tFkoXNqxWN8oCEd8U6iVGH9uOM4lNKtK5od5XFG52jvtdjiEA2XXUyo4b5Q4FlFXeiVq1N4aDsNmcLR0YYpyU3fyq4mjz9E/7JIJmc+Q3JYnxUSUcEB1eukIfSRQ9MmRRUQ6St45SLLmU293hTcNHIUGCb4lMiYmJqazWbOPXT7zv6AzxdEXWUDlHxZiqxzuBWSjR2hWgNBMx/Sg5GVz1XoWA3iknEcNOOZ7hvjMGJoNSf2XKySMFXaqu/NjEsZbWbitPXcEyk6eXeovpEOdXi0tbwdMtECWrSAeeEFRJGvgk+IboOwueP6UgVFyYQOs0DcDXCVTGW9SVYqRAwnEGQfXj4qdTo+MDBEIR7xYTHLQfqAHJ9RvVgTxIMVTXo2Q7l6vproF/HcD/3fsG/Xn10oBcKu/ORVcmyGG6kPKJjtWtDqGJPBdH+MqDuwAq7N+f69+6odXmqIacrWzpt8PA2c3YJz/yTWBuU+bZ9qwM9Oc0gdol6R8X15KLW80VrMvozHv4nn9LxV02a5YXr3AonlfEbMwMvbXEOJFkD6ZCMyU71Z08aRWILyCCkxtZujPkkfzfRy7zYnXlsEJE0649gp10Z7r5WwrZgQQ7MfdRaOoiwDyzLdbvx+r4kkPybWG9O9KSGYSzpQ62w+2uHIn7WH81kOY2Chrx1MtkilXVJVpMXqGXu6QjISv/5ngSBSIy0OQGoTX1Ew1iF6ZmUaxhrCvhvtzaiag1BwKaEp4W+4q/1WNa2jxv4HmGP0Wq1Ixd4eyF/TaLMp/FYebTJfYU4qBBhSraOY6L25LWUgk6go+jqvXOVMrmTqeJ+1+kbgYsxkpYVDIRYwwNoZXvfqWnFo5nsWb3fv/qtPOX+C0GkisisA5PkkZTE+6IqXdTwMwKbzw70Oc4CRIIV1w2T73iw6yuxfn9KCPhi9Gl8Ma6hB2K4wACEYHaavNqdm52fhv4HSDe4vSW5XPjBx3uf27NPfJzRyepcLLU5k44jz3gjoc05UnJHbaGBGGRwVCVr9TnPzK68Ia/98/ORCHxCUOKdp1SfG/+2mWLmc1+fzakK9lnfI0VjLmUttaYN11qCy3LmS79ZNV7vy/4Uwwd/1yZtEo2qq70lbS0qHy0CP4ANQEHSyEsr6emoVu5QAoMOdSrSWaCHUZ+7vH8iZC7Ex6YJVr+zeSuLAZixBy+Jrpqcsq8K3Jv07cAWNeZYqoPVEmJkG8YbdZeY7tNIMKp5poWAELYLb0M344rAKdh3S/g2Ya1p/S9S+Eof3rde96ol/rIVyb/DGtafDL+y0dCyOrzGHI4mXM9M2zSHLL/sZdUZIcCNaZ9zBHRqrVmhc+Z2XrWA5Qi7V6UDWOwOTjp0Xy9lio5cAupzrrzAVgP7vO68G+9uhJCqkTpzdez0Wbum6A6XfByWXAnvS6jun1/WB5N4vI5lNHrwqAjuQ1aeCt/7MyozIktNhSapvdodvgdS/2MgR8ugwe6dWH5Uz/ba8C9zwCWSHiGdAoW03NQhEcmK+sl58yiF5vLd6AqthfSouQ6TA5gN6Irgo5KYebpjRAjTjf8V+xC16XL7M/pF++Hk98XssaLMaucGQfa66mv8h+ViPkYBs6Pm1EGF8jDPsk/E5rrDazpwfnsvkuwV1aykkA/s68Cnfm650Ez1e/d4WJBWT8Xhm8UOr07zcoFcC/nFSzgI+lujcfB7T3B+JkEJqBShvFgr4/WwIjUh8zpV4Ps80HJ0soj7XAZByi6JKV4IgIgiG93iXk+wP7h1LhQzq4fpBNPpGfP3cf1Eth/tbiarFkaksnW1b2vVPQaLINEDRt3BEDBYbMDlP96WLhs4GgfiPwBvaaoEckfIX8+Cm7liBqetTn23F/r5P3m6gTB6TVDDhy82cNwvBXyJBvClCGPHduwVF6+aLV6Y38hG75rJEXu6i9Lib/eX6w9zxujgGhPFgyGaxWxuLftAv4ymaQyyFqrV4Zuwvvi+C0kFHa9YxYrvAOXk4ih1ar2u+bCVZ4N8ne9vfPRghjgyvIHAKmCRc+1dqW0n+OnidH/lGylGDA/qMI8ngjaa9b17PLVvQxg0boZkDSdIrhzaAPDMzfrj60zYHwlSj74MTJ4K1smfG3wPg3nPeAxncPO5jQ71w7k5K6wIg0YAEVYSUZ2AgAARXhpZgAASUkqAAgAAAAEAA4BAgAHAgAAPgAAAJiCAgAbAAAARQIAABoBBQABAAAAYAIAABsBBQABAAAAaAIAAAAAAABFbG9uIE11c2ssIGNoaWVmIGV4ZWN1dGl2ZSBvZmZpY2VyIG9mIFRlc2xhIEluYy4sIGR1cmluZyB0aGUgNjB0aCBwcmVzaWRlbnRpYWwgaW5hdWd1cmF0aW9uIGluIHRoZSByb3R1bmRhIG9mIHRoZSBVUyBDYXBpdG9sIGluIFdhc2hpbmd0b24sIERDLCBVUywgb24gTW9uZGF5LCBKYW4uIDIwLCAyMDI1LiBEb25hbGQgVHJ1bXAncyBNb25kYXkgc3dlYXJpbmctaW4gbWFya3MganVzdCB0aGUgc2Vjb25kIHRpbWUgaW4gVVMgaGlzdG9yeSB0aGF0IGEgcHJlc2lkZW50IGxvc3QgdGhlIG9mZmljZSBhbmQgbWFuYWdlZCB0byByZXR1cm4gdG8gcG93ZXIgLSBhIGNvbWViYWNrIGNlbWVudGluZyBoaXMgcGxhY2Ugd2l0aGluIHRoZSBSZXB1YmxpY2FuIFBhcnR5IGFzIGFuIGVuZHVyaW5nLCB0cmFuc2Zvcm1hdGlvbmFsIGZpZ3VyZSByYXRoZXIgdGhhbiBhIG9uZS10ZXJtIGFiZXJyYXRpb24uIFBob3RvZ3JhcGhlcjogQ2hpcCBTb21vZGV2aWxsYS9HZXR0eSBJbWFnZXMvQmxvb21iZXJnIHZpYSBHZXR0eSBJbWFnZXM/IDIwMjUgQmxvb21iZXJnIEZpbmFuY2UgTFAsAQAAAQAAACwBAAABAAAAWE1QIJUHAABodHRwOi8vbnMuYWRvYmUuY29tL3hhcC8xLjAvADw/eHBhY2tldCBiZWdpbj0i77u/IiBpZD0iVzVNME1wQ2VoaUh6cmVTek5UY3prYzlkIj8+Cjx4OnhtcG1ldGEgeG1sbnM6eD0iYWRvYmU6bnM6bWV0YS8iPgoJPHJkZjpSREYgeG1sbnM6cmRmPSJodHRwOi8vd3d3LnczLm9yZy8xOTk5LzAyLzIyLXJkZi1zeW50YXgtbnMjIj4KCQk8cmRmOkRlc2NyaXB0aW9uIHJkZjphYm91dD0iIiB4bWxuczpwaG90b3Nob3A9Imh0dHA6Ly9ucy5hZG9iZS5jb20vcGhvdG9zaG9wLzEuMC8iIHhtbG5zOklwdGM0eG1wQ29yZT0iaHR0cDovL2lwdGMub3JnL3N0ZC9JcHRjNHhtcENvcmUvMS4wL3htbG5zLyIgICB4bWxuczpHZXR0eUltYWdlc0dJRlQ9Imh0dHA6Ly94bXAuZ2V0dHlpbWFnZXMuY29tL2dpZnQvMS4wLyIgeG1sbnM6ZGM9Imh0dHA6Ly9wdXJsLm9yZy9kYy9lbGVtZW50cy8xLjEvIiB4bWxuczpwbHVzPSJodHRwOi8vbnMudXNlcGx1cy5vcmcvbGRmL3htcC8xLjAvIiAgeG1sbnM6aXB0Y0V4dD0iaHR0cDovL2lwdGMub3JnL3N0ZC9JcHRjNHhtcEV4dC8yMDA4LTAyLTI5LyIgeG1sbnM6eG1wUmlnaHRzPSJodHRwOi8vbnMuYWRvYmUuY29tL3hhcC8xLjAvcmlnaHRzLyIgZGM6UmlnaHRzPSLCqSAyMDI1IEJsb29tYmVyZyBGaW5hbmNlIExQIiBwaG90b3Nob3A6Q3JlZGl0PSJCbG9vbWJlcmcgdmlhIEdldHR5IEltYWdlcyIgR2V0dHlJbWFnZXNHSUZUOkFzc2V0SUQ9IjIxOTQzNTgzMjMiIHhtcFJpZ2h0czpXZWJTdGF0ZW1lbnQ9Imh0dHBzOi8vd3d3LmdldHR5aW1hZ2VzLmNvbS9ldWxhP3V0bV9tZWRpdW09b3JnYW5pYyZhbXA7dXRtX3NvdXJjZT1nb29nbGUmYW1wO3V0bV9jYW1wYWlnbj1pcHRjdXJsIiBwbHVzOkRhdGFNaW5pbmc9Imh0dHA6Ly9ucy51c2VwbHVzLm9yZy9sZGYvdm9jYWIvRE1JLVBST0hJQklURUQtRVhDRVBUU0VBUkNIRU5HSU5FSU5ERVhJTkciID4KPGRjOmNyZWF0b3I+PHJkZjpTZXE+PHJkZjpsaT5CbG9vbWJlcmc8L3JkZjpsaT48L3JkZjpTZXE+PC9kYzpjcmVhdG9yPjxkYzpkZXNjcmlwdGlvbj48cmRmOkFsdD48cmRmOmxpIHhtbDpsYW5nPSJ4LWRlZmF1bHQiPkVsb24gTXVzaywgY2hpZWYgZXhlY3V0aXZlIG9mZmljZXIgb2YgVGVzbGEgSW5jLiwgZHVyaW5nIHRoZSA2MHRoIHByZXNpZGVudGlhbCBpbmF1Z3VyYXRpb24gaW4gdGhlIHJvdHVuZGEgb2YgdGhlIFVTIENhcGl0b2wgaW4gV2FzaGluZ3RvbiwgREMsIFVTLCBvbiBNb25kYXksIEphbi4gMjAsIDIwMjUuIERvbmFsZCBUcnVtcCZhcG9zO3MgTW9uZGF5IHN3ZWFyaW5nLWluIG1hcmtzIGp1c3QgdGhlIHNlY29uZCB0aW1lIGluIFVTIGhpc3RvcnkgdGhhdCBhIHByZXNpZGVudCBsb3N0IHRoZSBvZmZpY2UgYW5kIG1hbmFnZWQgdG8gcmV0dXJuIHRvIHBvd2VyIC0gYSBjb21lYmFjayBjZW1lbnRpbmcgaGlzIHBsYWNlIHdpdGhpbiB0aGUgUmVwdWJsaWNhbiBQYXJ0eSBhcyBhbiBlbmR1cmluZywgdHJhbnNmb3JtYXRpb25hbCBmaWd1cmUgcmF0aGVyIHRoYW4gYSBvbmUtdGVybSBhYmVycmF0aW9uLiBQaG90b2dyYXBoZXI6IENoaXAgU29tb2RldmlsbGEvR2V0dHkgSW1hZ2VzL0Jsb29tYmVyZyB2aWEgR2V0dHkgSW1hZ2VzPC9yZGY6bGk+PC9yZGY6QWx0PjwvZGM6ZGVzY3JpcHRpb24+CjxwbHVzOkxpY2Vuc29yPjxyZGY6U2VxPjxyZGY6bGkgcmRmOnBhcnNlVHlwZT0nUmVzb3VyY2UnPjxwbHVzOkxpY2Vuc29yVVJMPmh0dHBzOi8vd3d3LmdldHR5aW1hZ2VzLmNvbS9kZXRhaWwvMjE5NDM1ODMyMz91dG1fbWVkaXVtPW9yZ2FuaWMmYW1wO3V0bV9zb3VyY2U9Z29vZ2xlJmFtcDt1dG1fY2FtcGFpZ249aXB0Y3VybDwvcGx1czpMaWNlbnNvclVSTD48L3JkZjpsaT48L3JkZjpTZXE+PC9wbHVzOkxpY2Vuc29yPgoJCTwvcmRmOkRlc2NyaXB0aW9uPgoJPC9yZGY6UkRGPgo8L3g6eG1wbWV0YT4KPD94cGFja2V0IGVuZD0idyI/PgoA",
    "https://upload.wikimedia.org/wikipedia/commons/thumb/5/56/Donald_Trump_official_portrait.jpg/250px-Donald_Trump_official_portrait.jpg",
    "data:image/webp;base64,UklGRhgYAABXRUJQVlA4IAwYAABwjwCdASo4ATgBPo1Cm0olI6wwJtRqqgARiWVuVDU4LqShIFYQ64oMPCLsWLswYvMOZGPvDnTQ7+Vt9lM4x5bwdftpN1kDFTZrgfAJRRhWXot/huMLERPRKBu31g/7g2t0CNP/6HrYyzKqHeTYobjfXJ2vG6Zi39Lp3o6rJq8fys12XCTRY3zqI5oe+rnH/b+QAwqyE12NptvxrjwJSi2Ozvoa/FQz/Vwt/PV4Pwr/ZqEGK6QGUsrlEkwHYnTAp8WV0ocaX3EB+uSUUgqjC/QYOokKxNJcjkoH8AYoJUmsWQ+rd9fhxgeURG5HLp38XMJnS2wmxfc7Nd4BwBzV7n8ArdtuSprRgDB67AZsnVWDB50cxZXNOC02kvrSU1hNVyVF3r1hLjP4N09D4ku1BqN9ktDZV1CwYpDhX/BmClZAcvca6E3ftLEqZ3s9qPzmhJ7BcpYqtb8hxdyVMuODS5EyqksnTcFq0Pf+NEtDKHaXv7gD4U30uvWR2MoCdTPhnas2em+vq6E3ezArmjEiHtR/aO/IgdvX5nZYpV64+WKK+fBOf+bhAGWwqNyqFoPk4SJO7EGPHgB9CoRyXlmC66z5v4a8TGfL9yG5qI29Ouzv6fWSHN0HBk6dJmFLq+R2BEDBrh2ES5OdQ/Kv9JhGSeD6sQbpJ19FkKjzyWf7gXjqtHPUyH1YRd+SsU0ctPtiXDiH0GWBnlNLXhfny3eGNPhXcmf5F4V9t2wABtUcnXXngaY+ihisI8SBc/dyBy/ZGdY9xSWcntz/hFAMUfKR3F8ZpTIJbyUyv2Odsuzw2zjFYPsA+EaUXRcDts89XVtqMVxrIPQ0gzMR5MDtLY83RY6m+DAtsotAgteI9GiTHmmYRg3Mz0LQFOWnRZuW6CN8vXUtUa0ByaQhvi9lP1vjUaEsskSv+C0rMUL0rGkgMfVVJpo5D6OTWvOaDyEoFOVDdaOy0h3LhpAZqGmc7g7g/Y71WgSLVTZCWw+dp/CggVNpNo8r4bnz9eLZTfXmIQWsYKsK0nw68a49gO2PSpcR1QNGwc9MlLCBzx7IcXTVEqRN0cuVLO//QbH/7QpyHT8EgEDFjllwDXJqJxQAV+sJ5xbJrd9J7to6akgA7WHPEVRkHPhzikv7I22fBFkh8NA+aLKiExKMV8QLwikGtdNnvaTMwVcuzx1m9QuKcZoN0rtyl66GXmbGtM5sNZHUYjeVMY/90La/Tnu6bloQlyRH50mNO+7ID4XhZF9HKP5yuSUwrM53IzPdmGwbPFHoaGoLiaxkLu48njrE4HQtdcbqywOqWGdoAmoozZome122fw9LP0jUGNjNxaIzRhqXrvBbFlM4gAX/CbdjNcQc7MVCfT62n3Rj1E2r7fqhbfc+uFnVDFf7qImwfyeaM6GthoK5gR7D66k6Nb5ij/FwKtacmQ+l8Y4dbrNGnZnOPiXo/BQQSyLbaVrw5TDS6ys5Ej1InE0quglwbIzf5l4sK1DZbZTjB9ZlixcCO7uZ6QA/xL9PRPdq5sq+4ELDxn89Rmm7zGzATpvEAP7w98u6TN9cSNNd33SP1MIuxcRpg5RMY3Bml5nPZIkB7rec+nU/cpoSGWBg6MQR60Ovd7Szh4JQi5ILI/+jtJha859SG25esS6wPQbzPghaacQjsdAmAMM0yJ3AdfPBA3R7SeClGttHjRebuHHsV6gWYyYnVFmj0f7lHDqatOQ79u4MBi+znrd97OITutraPE3cimceHLAsHWMyO2bjTonFpHEdC4OitFPgYT0Ni4TCwfyM0jO7PXc6ixRg6MOeR1GCbZNGpsWYeh7Ermd4rd+f7dCChwlWFGR1d6HYSzgT8tMbLwe3XjJ3xWLsCx8Iwc8CvTqBjJBUCRzUnsfiJyaK9GwLztaqSmiHnOgCfieNUTM3op6GzUmtrQf5DgZVIviip22779CB9mSZwuq62YvPkMzbIn2e+gHxwO6QoURUI9wTHg23DnZzlkBWnXdUCcDwncvHmGB3Ba8xw3fez1zujlLg1I2NHfi2SrAOiXK84dKBsu1L8N0Yz+dcW5OTT0frz0FgBrw1Tnr6vTfAkKziXRDD4DHfGlj6jYYg7VUvb7enrgE86ts2AcKJacg5mR0iWavHZwiptcEL/LEPj8MvCNGjdml44conIkCxhkEr8qXg0EpVRc1CWD4BPIfeH3jgVjQR8OPdoCod6b3xz2uHs5A45RFwrVMewWMe0/xcHkwzZJeliBhArxam+TKAfhfFUBZNgyqVX0X2zjxemK7Uw2VMz75LZPVXoZe4TY5TdUVCtrgIEIw4PDF/cTM79V1dtqNmW5u3W2ix5HNwVyPxWl972k0t6Ayq6eSPfdTmqe/2MkucU4UrlpQopJWKPrKTa32Y169+9t0qdpUQD3e/xILmVuCA8CPJHN8gpZ6TfXQ4gx0AzYY7ifOhm5IFmY0LNTaYrzbEFqk73mNf8v3A7AWTgijsYqcvL8e1NilS5tR90nTcR+a7uEe9vZ/gpLVJi6D8NjANeEYkpkdpOdY2fH/6m7kxTIpbXrCEvyoAfGudPLQCoD5de+/pTP1mLHDTMoVgpUuYPdw27w4cFUKDeaPy8dn/dxthrpi+lpibdywKoiS1ljRj1S2RNNkkXZtQqK+e1Heg1QGGPXRvbQ5Hn6yviLv31N2fn5IxP59iCfHxTS1xT18dO1LVaxBGqIVugkKGOVeViekbUljeWe5fRXbqKZ3iYSi2LYLLHLyUDmwUg28ryZlq+IbAJ7WyLSuVYUELsu3mjPx6T7LYa+AwWFatbsJlg6cED6K9lu6GdwZg9fgyg1DC0vaNVXrcE76q3MhjVay1zdQCblZBHamS1vY6ugkLlIELgMjelSeQMK/snwjsOxWkgmv5ARu539XhdNUYsX9jpTBHRgJn9OvRPnQ0zl/n3Vf4CpfhLVy1mG8iTFNlXUsRTI3d3pbp+tWupsZKacvpuozCh8IpEvDJ0Pc/YFytcJ5eT0zRX+DpNNDWpLz/w0B4wWuaj5ma008ygKHI7A9Y4FecUFjt0zJhK4AmD+QcI0s2JeXtSfTAakp6K9xe54dYx9mDwCtcy2s1NIlxBA76lhCY2fCdQ5vods3GQMGelQQvj8y5yBf+1zxAy6acoITR4swt3QbvKBf98EP5sxnI77ZuJbAz6AVfZWYNVYfN4BZ1HGfCNMFLRDclVOfgIXc+P02t5f7nNDf9jAkWkVW8gWgzMawEe1hZbaEEXMJe5CXrfozjw8v5mVXAgo27CUGDWzphgVFix6wj12TgKuOkven8A5mGuOyrK/jRWiKADl0HKV5ah1Fz0ETGQ5nZfVyNrd83aqV9qex21d468I+IeGy/qQl4Es42LA3hq3A3tZwFGBulaCy1KK8iVpQEN8qs5BME2z7INRBb2KA0ZULXw+BHtgIM67RRyj+SezNM7aao5sRN+KetcRUr40twilDSsIOTGq0DgkeYcX0zJnHarZi52AWJPyhU4vD7hcQWgF90mjfXAbxeF8q9GDO1N3AbAt/O09XoPo+v+zUczBHPq0fQMv0y+0P5aD8NVxN9qPGDLAc51MaVTmuaaR3FYcXVV1jY2plpmID8zOws+c812ZC8dAnKjrePsPnsyfj9f7hX/l66CVTnbF5Uq8mbaWAAmvST5fgcLC3EIpAYtiVq7BW+Eki0qSZJaVeTzWkwI+/Qf6+vsQNzYp01xqgwzHguGQQHZYCuSwP3EGtXgN1lNzcWxUs2SEZVQSp7x3Zf9lhYo2M+47zIzMV5Ns7BewbTATuBrRKLtU84IwNlYgDNgxqUVck1xdbcmeDq14uJNAL6aGnY1Izixu82f87vuB118H0COvfqytdsoFPa4PFv1nkfKAiwZ3h9yj41Z2+Ai06zSR0ngfj+X0bccEghLHOM+EpIwLBBP+IJWpMo9+TjiHIrFkWVwUOHxVNEI19ZkrwJO6M0zzMbWUDrBSKSTovXYyqGn7pzBZUasKohtsxreN+/zGjBhI3Z6Ut17EP9KzHWIA9GqzjqQQh8DZUwVAHK0G29b79CyjLhirO9VCz+8/kVGMoOXDu2ymj5L7NLEiBnhkvX8H4umUFFR06V4B9aVbLQXMQsmbbm2vPDwgm911BN776KGESsP+NaPh9VlcrY2ZD2xoVedSteXKJRYYfrSck1ZCPeDnpQ2GdiWZ2iNnCgkeetDo8g8z01TBk+k/QAjlkKrHijJn3nNHXoEWHHcpqy9o0oQ3yjdkYRel4jjVkVog5VVLY5y3eIAbNoj4tHDogQDATpmiSYc720qWk6KkNp2c/nIP6/om0Ztaq5Byb3MpQL2aaWBDOyLDYN+uBDFrjqRcB7B6YA3/1jhtJmkPKjLof8NbNZUv0FjpIYHht8HkYg8KD4cOzy0uoDsnFVbSWLQ8Nyx3JyxBBZIiGaFQWwPFBuYe2EVyh5aJ9VbdrUsUjGnMg8EasFVIX6zmivAZ+n2ZLcw0BBrzxOu0XXfIVYuk3CjiZaoXIv5v0qBH4NT+OUi3ruLkgCBfno7qz2cjqbRQ6rFH3QCZNQaDoehNcd9SE3CzM9ADorn8vajC0pB9waCPxFh0dGSN5AeUY95sA396Mawta5K+8PrCbonXjX57ujHVUK+5pteKSFTpuACJCR4OvJkOShxOrziB98IfXKW+pMPxbDM/uD4mjzR6UH5IFyhOzi1uNcQhCMTGrmx5TQv/2sdB8MUZEID37Gz9im89ja5Zfg2eFSlAjlDk2GUXDPPXoRaFRURhITfy36QPmYEX2X8hWHyf+PWbKOmUNFE4sxzf2+BK6Ku7afVg33MOn9zMFkTl0ajsiVGIEsXIy6YSJ2kCixXXDVBjy1ONq9g4H/mo10SUOthBWVVYEr9PWRnno2QaeLdW5JpHWL1tSoKLIK1WoaEW4s+gTf+th+icFMlq042cQ2qnEaZWnNIc169jtfAaKeT/FdPtBf10huObSFBOxyFK6hGS5JtBtqTJTxYGRSJGRho+XXprpna/90toUqMCcEE874pVjHSIzl0Osz9LOTImD/414tUjW/RGsZ39pp3RXUfOMugRzdshjsReo4dlXp3XQHh3xLf/aIsus0gcyx9C9MgdouYa6eSnlBFih4sFGUH0DlgJNoqdpzAjgRxNjFnOviG4dAzWl9uF0hPuP91JtGks+uvifEahjFV2nR5p8SvgDCPkI0fgf14tN0HSfMDxFQtjeDuum8yXVDQcDReLv9lIFderfa+Asvqyo2/K/qCHaVVaML+PQ5C7I0dX2BctsuNL7EO7hOejAWFmAA5nrhzY7j7UmV+ZAWotcOPsrXRnBHhvSFbhteD5HMrJnackK/ZG3qDcS++KwJIGbIUmh/2xMrPh5PU0Bixz+PP5NBDUTx31WTR9EGfctS3EArTOhn+UL9VJgdGfaHjCb9TS+ai7QqHV1vNCBtPu41LX84qOS++IhqwrEy9vCcnefCdhcgXNWumnJaB7foXD25brmMCFgpG5c2y6/BrIye1B/67pf+HN443x+/oeDFN9rH5YcdzW2uTw6jDXqE70+4LIL9WFIuKzultrC4TOxN/Bp737wISqqkhAncLFUmJHgFTC1mVkaEDSdRa7U6QYaAhtQNZbMamVNRU7ueUd32t6i9BVAsTI1zMTQWyehsAyimHEW/jpozUqIaAF5dTBNJh5aMDaT3/M3izvw75A7QeoROPqKNV2YHhtgMQ6YZHFHdUdvzvWrpDctVKZgHvHH81f+IhYergglXKXFIKVzDD0MkvmS34tQJmlwE4cRRwyy2VSNShLvB+FM0qYvJ5qzqnReUF+eTfbK1Iozb8NgISGDnNn7PDsRBioSpGDpiRuRkedQ9miag574WZuu69D1sJYfPGe+O88nQh0xwC/3LIyyp0PUpD5/F95f119V9axg3N+HG0jXI8Xq8hPFHwyXAf73ypGsfELIDsetHZFAC4aav51wtbUCxfy3ptQNWMQZF+fOE8LeIAzTAxdtjJZZ/ZDcNhMBtaAAIv8uZ3+2qKHIM+Wlg0bYyRjziCpyDXxo5sIe/Mb+/2BjSQulE38cOqEdcyxGsyOa6ZNe/+RvIN7qQK13FkjPIC5ngAKl35YizTnxi02i8BR4xGzpiFy1FjVt8oqAt/zl3hFGKXzcL2ncSFkUS1ceIKY7C34EWg1IJkvxp1aNitdvtxQVbqbyCLuyg9pNJU37F+y3YE/ex5+AbYUMcMOcCZSvQWF94NVhIyQmiIVAr1Lg6WxuSBk3zrB+ZKqZeNE0FETF/rXhyzSEc+t4xunOIwmO5yAEY5Aswv29HK+bcjsSBCfYCLO3//bxMRQNpbHIoiqM0KUB43CAFWgMGHJ7LjTDeAetb39EUJW3EFvH+JSfwtcWXd4wr571G0cfJ0pWfbGMR+nt6WdxGMrARrt4NhsMgU4T+5+wZ5eiDJLeqBwVoYSMSBct+zg8wWXrm/O9c1igTMc2J5C+SvWYPsm4nj3XHXYLpDGzg6IFlvcsed46guXT92xzdAOZ0K8BrvYrocQcBiFM0B3RLKlhJmR86qjhlGsnQmI29zp3+uuvgOMWLf/DYQeH/6A9FyUYuT4bh27OyZ5oYjP7Hn8SEEFqxyHXpjYjImsTIBClBnxNXPtykemECaEDeGJK3mZAupfTqt4Oul4JxmkBX5uN3gliYYwSEn3QhcOL5iq7fpIySpDBZb9BHGxAOpO1MtzxlifSaAkjN31TkT2AAWJiKEpwEB0nJMsK6Z2Jn+T68qouqH/XUEXhsNXiO3yyLcnzIL7rKmnnGXFcUM5VZA7rsVcYpl5P4XpZb77X+FJ3SD0LvyUjJ8rim4Z48de2V/z+OUoLalA/k0KeRFvXhnsY14eixYtgo+L69tzte4xog3ZkaLD7puio7a8xRTH6NnineUmvH839Rr6bk5OezMrkyqg/n3MlBjYaLTcoZJc2kDRUbXTIoB+XX/O3U/mH26WNnSioV4DJbbwdj2AeMvIcnCDNcFjk7K3zpTPlaOSo8subqOoCzH/aVlKHacJ+LH4/W1gt+5qS0Fxpsurj/QPWwrccxGp60KEJgmZFHQCx5AfvTHroS2SincKARP3yBuJYakU+Qu5tlLIo6Wg/usdVHPvYq+Z3A6KySYvU7SxuDy0jqDyBzfTkSHDU4MEXIZ5MFV6ix4qY17v6uLAp8B1wjS60oiLCkQJbF8QHJmpXKDNfNc+LIbXt7h3q2R1pGlkKgamwEn+B/H6egyZhN2LakS43jTINnZFPaBZSma0scJBemIFfBKXuM3/uormolhkoC+VPneFuI+eeQYUZaHoEIRvKzOLdRJNvRymnERyEPNXDVTGfBJQrXWN0zvfpgk0bbr6klMRd/+g5O7+wR68E5S8MnFobWDdvMTt71RI0HRRjOtameisnikAzzoFzXEns0AA+PUy+Ozim4G5mpuGriQeN6S1tGST0dYZSGw2EhesL410zJUbAkl1zIXwpxbmiHFcyn/6qvb3pcwxaJC80NjQjtmsZvwd1FGlRCNA3+laxHh5fwB/xSuUWzcaPkiZ6oBpacFZkBp4w4IoRa/4Z76sK0BUgO73YX+lNh0Iy1+ZIfZMPGGTj0Up6gMY9uhJlrlGo9jMtrMQWOTk7Y29lkr22ifWB7E9xbzJyQit9iBq06CAlYC7VE5T9MBs26wN3/zTjlJdGiZPrc4VgXy2QkOW0F/Wkw10Bcb/mu5sYX4tabGYbXNrnK50w3NtSQH9sd6tsrnv87lc41fc9RY6BFkT3/5h+sbCFzckRru+/n7Im9xHOnK0z2WHEv/F1TZInh6rC4aMxzuhGQz4CFmgez9gAH03Na4V6fG7oX4bE9tyBkc2lhvHM5LQyj61l8/RzMgtjksgcijtA2aWsyzKe8P3C/I+XKD1S3+tCBOPMKdumEnk2d68hxBaUnmTACmhySkOhhsJa3R6Nvwux/VPlPuexaR0+yfpQgTnIvthOazg/GhFOp5+Sr3a9UYLN4Rjk3lsdIITSEarimmOr07lxjUsMmKJMhiMFd1Zh3YEnQpR2KqfnRkNbILCPXVKVFPAG9d4a21a7Hd3fojNYFpoYX8onV/qRBPo0WxgLZGAFc3bZOkwER8EBXlc//5beBJk2cz1lPt2L2Gs5qPBiLjCUPqG2TDvljp4zagyzpLi0Rcri24W4eRBn0nqoYqvPe1/G2Ii1xA6pQ7uE34U4dpD5DAhg3kw1aSy0EAaEAAAA=",
]

# 準備長條圖 線圖 總篇數 總次數等數據
list_freq_daily_line_chart = []
list_freq_news_category = []
list_sentimentInfo = []

list_total_articles=[]
list_total_frequency=[]

weeks = 12
cate = "全部"
cond = 'or'

selectedCategories = ['全部', '政治', '兩岸', '產經', '生活', '社會']
# selectedCategories = ['全部','政治', '產經', '生活', '國際', '社會', '兩岸']

for query_keywords in list_pkKeywordSet:

    # Filter news   
    df_query = filter_df_via_content(df, query_keywords, cond, cate, weeks)
    #df_query = filter_df_via_tokens(df, query_keywords, cond, cate, weeks)
    
    # Get line chart data
    line_xy_data = get_keyword_occurrence_time_series(df_query)
    list_freq_daily_line_chart.append(line_xy_data)
    
    # Get bar chart data, total articals and frequecy 
    cate_freq, cate_occurrence, total_articles, total_frequency = count_keyword(df_query, query_keywords)
    list_total_articles.append(total_articles)
    list_total_frequency.append(total_frequency)


    # We select these categories to display: ['全部', '政治', '兩岸', '產經', '生活', '社會']
    cate_freq_selected = [cate_occurrence[cate] for cate in selectedCategories]
    list_freq_news_category.append(cate_freq_selected)

    # Get sentiment information
    sentiCount, sentiPercnt = get_article_sentiment(df_query)
    #sentiInfo = '正向:{}%, 中立:{}%, 負向:{}%'.format(str(sentiPercnt['pos']), str(sentiPercnt['obj']),
    #                                            str(sentiPercnt['neg']))
    senti_info = [sentiPercnt[p] for p in ['pos','obj','neg']] # 只要取用pos,obj, neg的情緒數字篇數百分比
    list_sentimentInfo.append(senti_info)

# We need all the following data to display on our frontend page 有一大堆數據要送到前端去展示啊
pk_data =  {'list_freq_daily_line_chart': list_freq_daily_line_chart,
           'list_pkNames': list_pkNames,
           'list_colors': list_colors,
           'list_photos': list_photos,
           'list_freq_news_category': list_freq_news_category,
           'list_category': selectedCategories,
           'list_sentiInfo': list_sentimentInfo,
           'list_total_articles': list_total_articles,
           'list_total_frequency': list_total_frequency
           }

# Step 5: Save PK data to csv file
df_data_pk = pd.DataFrame(list(pk_data.items()),columns=['name','value'])
## 存成csv格式檔案
df_data_pk.to_csv('pk_person.csv',sep=',', index=None)

# Load Data 

In [144]:
import pandas as pd
from datetime import datetime, timedelta
import re

In [145]:

df = pd.read_csv('./news.csv',sep='|')

In [146]:
df.head(1)

,Unnamed: 0,item_id,title,category,content,link,date,photo_link,tokens_v2,top_key_freq,summary,sentiment
0,0,_20250327_1,台股重挫308點 失守22000關卡,焦點,美國總統川普準備徵收汽車關稅，美股主要指數全數下跌，台積電重挫4.09%，台積電台北現股今日...,https://tw.news.yahoo.com/https://tw.stock.yah...,2025-03-27,https://s.yimg.com/ny/api/res/1.2/3qEveVGKp070...,"['美國', '總統', '川普', '汽車', '關稅', '美股', '指數', '台積...","[('台積電', 4), ('指數', 3), ('月線', 3), ('美國', 2), ...",暫無,0.36


In [147]:
df.shape

(216, 12)

# Step 0: Filter news articles using the following function

In [148]:
from datetime import datetime, timedelta

In [149]:
from datetime import datetime, timedelta
# Searching keywords from "content" column
# Here this function uses df.content column, while filter_dataFrame() uses df.tokens_v2
def filter_df_via_content(df, query_keywords, cond, cate, weeks):

    # end date: the date of the latest record of news
    end_date = df.date.max()
    
    # start date
    start_date_delta = (datetime.strptime(end_date, '%Y-%m-%d').date() - timedelta(weeks=weeks)).strftime('%Y-%m-%d')
    start_date_min = df.date.min()
    # set start_date as the larger one from the start_date_delta and start_date_min 開始時間選資料最早時間與周數:兩者較晚者
    start_date = max(start_date_delta,   start_date_min)


    # (1) proceed filtering: a duration of a period of time
    # 期間條件
    period_condition = (df.date >= start_date) & (df.date <= end_date) 
    
    # (2) proceed filtering: news category
    # 新聞類別條件
    if (cate == "全部"):
        condition = period_condition  # "全部"類別不必過濾新聞種類
    else:
        # 過濾category新聞類別條件
        condition = period_condition & (df.category == cate)

    # (3) proceed filtering: and or
    # and or 條件
    if (cond == 'and'):
        # query keywords condition使用者輸入關鍵字條件and
        condition = condition & df.content.apply(lambda text: all((qk in text) for qk in query_keywords)) #寫法:all()
    elif (cond == 'or'):
        # query keywords condition使用者輸入關鍵字條件
        condition = condition & df.content.apply(lambda text: any((qk in text) for qk in query_keywords)) #寫法:any()
    # condiction is a list of True or False boolean value
    df_query = df[condition]

    return df_query


# Step 1: Sentimental polarity score計算整體情緒分數(影響力)

In [150]:
# sentimental polarity score
def get_article_sentiment(df_query):
    # df_query = df[df['tokens'].str.contains(query_key)]
    sentiCount = {'pos': 0, 'neg': 0, 'obj': 0}
    sentiPercnt = {'pos': 0, 'neg': 0, 'obj': 0}
    numberOfArticle = len(df_query)
    for senti in df_query.sentiment:
        # 判斷文章的情緒極性
        if senti >= 0.75:
            sentiCount['pos'] += 1
        elif senti <= 0.4:
            sentiCount['neg'] += 1
        else:
            sentiCount['obj'] += 1
    for polar in sentiCount :
        sentiPercnt[polar]=round(sentiCount[polar]/numberOfArticle*100)
        #sentiPercnt[polar]=round(sentiCount[polar]/numberOfArticle,2)
    return sentiCount, sentiPercnt


In [151]:
query_keywords = ['川普']
weeks=12
cond='or'
cate='全部'
df_query = filter_df_via_content(df, query_keywords, cond, cate, weeks)
sentiCount, sentiPercnt = get_article_sentiment(df_query)

In [152]:
sentiCount

{'pos': 31, 'neg': 6, 'obj': 3}

In [153]:
psentiPercnt

NameError: name 'psentiPercnt' is not defined

In [ ]:
# sentiInfo = '正向:{}%, 中立:{}%, 負向:{}%'.format(str(sentiPercnt['pos']), str(sentiPercnt['obj']),
#                                            str(sentiPercnt['neg']))
[sentiPercnt[p] for p in ['pos', 'obj', 'neg']]

[78, 8, 15]

# Step 2: Count categorical frequency and occurrence(number of articles)

    Count how many articles containing these keywords計算各類別多少篇文章提到該關鍵字
    Count how many times these keywords were mentioned in each category計算各類別出現關鍵字次數

In [ ]:
# **計算各類別多少篇文章提到該關鍵字
# **計算各類別出現關鍵字次數

news_categories = [
    "全部",
    "焦點",
    "娛樂影劇",
    "國際",
    "政治",
    "社會地方",
    "財經",
    "運動",
    "玩樂",
    "品味",
    "遊戲3C",
]


def count_keyword(df_query, query_keywords):

    cate_occurrence = {} # number of articles
    cate_freq = {} # frequency of keyword
    
    # 字典初始化
    for cate in news_categories:
        cate_occurrence[cate] = 0   # {'政治':0, '科技':0}
        cate_freq[cate] = 0

    for idx, row in df_query.iterrows():
        # count the number of articles各類別篇數統計
        cate_occurrence[row.category] += 1  #   {'政治':+1, '科技':0}
        cate_occurrence['全部'] += 1
        
        # count the keyword frequency各類別次數統計
        # 計算這一篇文章的content中重複含有多少個這些關鍵字(頻率)
        freq = sum([ len(re.findall(keyword, row.content, re.I)) for keyword in query_keywords]) 
        cate_freq[row.category] += freq # 在該新聞類別中累計頻率
        cate_freq['全部'] += freq  # 在"全部"類別中累計頻率

    total_articles = cate_occurrence['全部']  # len(df_query)
    total_frequency = cate_freq['全部']


    
    return cate_freq, cate_occurrence, total_articles, total_frequency

In [ ]:
query_keywords = ['川普']
#query_keywords = ['馬斯克']
weeks=12
cond='or'
cate='全部'
df_query = filter_df_via_content(df, query_keywords, cond, cate, weeks)
count_keyword(df_query, query_keywords)

({'全部': 162,
  '焦點': 59,
  '娛樂影劇': 0,
  '國際': 33,
  '政治': 2,
  '社會地方': 9,
  '財經': 52,
  '運動': 3,
  '玩樂': 1,
  '品味': 1,
  '遊戲3C': 2},
 {'全部': 40,
  '焦點': 17,
  '娛樂影劇': 0,
  '國際': 7,
  '政治': 1,
  '社會地方': 2,
  '財經': 8,
  '運動': 2,
  '玩樂': 1,
  '品味': 1,
  '遊戲3C': 1},
 40,
 162)

In [ ]:
# query_keywords = ['川普']
# #query_keywords = ['馬斯克']
# weeks=12
# cond='or'
# cate='全部'
# df_query = filter_df_via_content(df, query_keywords, cond, cate, weeks)
# count_keyword_via_tokens_v2(df_query, query_keywords)

In [ ]:
df_query.head(1)

,Unnamed: 0,item_id,title,category,content,link,date,photo_link,tokens_v2,top_key_freq,summary,sentiment
0,0,_20250327_1,台股重挫308點 失守22000關卡,焦點,美國總統川普準備徵收汽車關稅，美股主要指數全數下跌，台積電重挫4.09%，台積電台北現股今日...,https://tw.news.yahoo.com/https://tw.stock.yah...,2025-03-27,https://s.yimg.com/ny/api/res/1.2/3qEveVGKp070...,"['美國', '總統', '川普', '汽車', '關稅', '美股', '指數', '台積...","[('台積電', 4), ('指數', 3), ('月線', 3), ('美國', 2), ...",暫無,0.36


### Count word frequency using regular expression

In [ ]:
content = "台北市副市長川普今天表示，幼兒園確診老師案目前沒有新增確診案例，但確診的孩童Ct值開始下降，川普...川普川普..."
keyword = "川普"
re.findall(keyword, content, re.I)  # re.I ignors case

['川普', '川普', '川普', '川普']

In [ ]:
len(re.findall(keyword, content, re.I))

4

In [ ]:
query_keywords = ["川普", "確診"]
content = "台北市副市長川普今天表示，幼兒園確診老師案目前沒有新增確診案例，但確診的孩童Ct值開始下降，川普...川普川普..." 
[len(re.findall(keyword, content, re.I)) for keyword in query_keywords]

[4, 3]

In [ ]:
sum([len(re.findall(keyword, content, re.I)) for keyword in query_keywords])

7

# Step 3: Caclulate daily-basis number of articles which contain the query keywords

計算以時間為基礎的被報導的篇數

In [ ]:
def get_keyword_occurrence_time_series(df_query):


    # end date: the date of the latest record of news
    end_date = df.date.max()
    
    # start date
    start_date_delta = (datetime.strptime(end_date, '%Y-%m-%d').date() - timedelta(weeks=weeks)).strftime('%Y-%m-%d')
    start_date_min = df.date.min()
    # set start_date as the larger one from the start_date_delta and start_date_min 開始時間選資料最早時間與周數:兩者較晚者
    start_date = max(start_date_delta,   start_date_min)
    

    # 設定時間欄位'date_index', 次數freq欄位，將每一篇的時間寫到'date_index'欄位，將freq設定為1 ==> 進行groupby之後就可以合併日期與加總freq次數
    query_freq = pd.DataFrame({'date_index':pd.to_datetime( df_query.date ),'freq':[1 for _ in range(len(df_query))]})

    # 開始時間、結束時間兩項必須也加入到query_freq，計算次數時才會有完整的時間軸，否則時間軸長度會因為新聞時間不同，導致時間軸不一致
    dt_start_date = datetime.strptime(start_date, '%Y-%m-%d')
    dt_end_date = datetime.strptime(end_date, '%Y-%m-%d')   

    query_freq = pd.concat([query_freq, pd.DataFrame({'date_index': [dt_start_date], 'freq': [0]})])
    query_freq = pd.concat([query_freq, pd.DataFrame({'date_index': [dt_end_date], 'freq': [0]})])

    # 透過groupby就可以合併日期與加總freq次數
    freq_data_group = query_freq.groupby(pd.Grouper(key='date_index', freq='D')).sum()

    # 'date_index'為index(索引)，將其變成欄位名稱，inplace=True表示原始檔案會被異動
    freq_data_group.reset_index(inplace=True)
    # freq_data_group = freq_data_group.reset_index() # 這樣也可以得到同樣結果

    # 只有頻率次數值y, 沒有時間變數x, 計算相關係數時會用到
    # freq_data = freq_data_group.freq.to_list()

    # 有時間變數x,y，用於畫趨勢線圖
    line_xy_data = [{'x':date.strftime('%Y-%m-%d'),'y':freq} for date, freq in zip(freq_data_group.date_index,freq_data_group.freq)]

    return line_xy_data


In [ ]:
query_keywords = ['川普']
weeks=12
cond='or'
weeks=12
cate='全部'
df_query = filter_df_via_content(df, query_keywords, cond, cate, weeks)
get_keyword_occurrence_time_series(df_query)


[{'x': '2025-03-23', 'y': 0},
 {'x': '2025-03-24', 'y': 1},
 {'x': '2025-03-25', 'y': 0},
 {'x': '2025-03-26', 'y': 1},
 {'x': '2025-03-27', 'y': 38}]

In [ ]:
query_keywords = ['馬斯克']
weeks=12
df_query = filter_df_via_content(df, query_keywords, cond, cate, weeks)
get_keyword_occurrence_time_series(df_query)


[{'x': '2025-03-23', 'y': 0},
 {'x': '2025-03-24', 'y': 0},
 {'x': '2025-03-25', 'y': 0},
 {'x': '2025-03-26', 'y': 0},
 {'x': '2025-03-27', 'y': 7}]

# Step 4: Process PK data

In [ ]:
# 定義PK對象
# 通常會多個關鍵字代表該PK對象，['柯文哲','柯p','柯P']
list_pkNames = ['馬斯克', '川普', '柯文哲']
list_pkKeywordSet = [['馬斯克'], ['川普'], ['柯文哲']]

# list_pkKeywordSet = [['馬斯克'], ['川普'], ['柯文哲','柯p','柯P']]

# 線條顏色
list_colors = ["rgba(0,128,0,0.2)",'rgba(0,255,255,0.2)','rgba(0,0,255,0.2)']
#list_colors = ['green', 'red', 'blue']

# 人頭圖案
list_photos = [
    "data:image/webp;base64,UklGRmAWAABXRUJQVlA4WAoAAAAMAAAA4QAAmgAAVlA4ICYMAACwQwCdASriAJsAPpFCnEulo6KxpBDa+jASCWdt/IkYYM03ONCiTs+VIMbuwWWKAH8i/xHowaEPq39mfgR6QH7h+xQxVpkA0o65DXBCpBN8pXOp7PNbdQmjzx31hzv3guNEZFeUzndlAnBxndff8JntJ27pGPEW8ZXTvJjbks44/zbRD8SKcg4rBvSwtz6HKhwVev8fMCWofd9mHIZKXowkPpgwbWTeylxMve5SbAJLfBs0V0n8qlx8wcTHO0aqIJxyv85WgNz+0rMvaGQtGdEYuM8Z5DVlgXI0kf6kqFQATXmmLmPgeXA78c+jIvaEjNkFxHYx/baDg5tPy+H5YF1+L5z5pRtfMj3qQ0sVPqGZGtrGUrWhTj9F1Yiz/daIL88LsU7QBXg+23TUoLSntvbGFfUIOAXF6hs032rgIEYxV7mYVwYiUGQbG3Pc+iv6y1xA8FXfua/lZUmEx509lrzF94D/bd8N8UtDyGvXl2/FdvRR9HyDHbNBTV1Z93SPOD4LuGCQuSGra+ev6LhwbA4I+kLOWVOUIX1NxXe6UJj8gwSS5XYtJJoQ71xhEByHrAcycc4uLvZf29bG98+nx0Qt8e4OC/JPCrxG7rT9+zzyDTJVDAlU9EK3XTNkazuyvOGABjJPkgHvhl839Fm46MgXieF2QnSsvhlRjIx6+z1+4SvXKfi7+aRuMKWZkZ3wuxx1MgbNJoTS4seSmDoPEu5+Nz/3FGxYAP7qvqiWV4wML+DqzerCtHwmAg/ZQVwEjzzlmqLhj3dKu940x87bQbLdeGnSZGymF8UO1c8IfVeH23LkXJkxoOUdsDhwYjIxJbNCS0C5BFC18j6BxJz2F1epAvZSLkLf2bTRQ4b6d1NlegcfKlh8Ads/wcMN61RhlDWCR4rkuiofN/aCfPhsKFNkFv40EV8l6Ep0X99vH12aAJcHWVEq14xsuxKspuUYP6etI15sX+GMPEgXbqVwyF3qscm9mD6gNFu6bphl01ZhGznLsI2Wkyrua2nW7QgG2ILeMwDeP+if3XefghP/7rFzULUM5x6q/0jLlv+ji4OCE/5ivn+volZ4M2YaLr+u2252i/hfFb/c6AB004JXpw+ClsxR80QrwW2M/r9Pmrdgw+NBXO0nLXY4zpIouOW8p2PfHhwkEgaRP2YXbFgBtRm32FzTM8Fl58k6fARUIuH+lCeGan2biKiiArxQObp+Pegwy+7EgKsb2THO156VkKAPiPKQjfA8PU9hnuxSTWl+zUXUED+46geasqG8asyfzPUCIZdSpiy+YrZfiBZ4Zu7kMg5ABetH4OwY7a6Et01JNRnjSlAqLUuWYBwWRAQID2SJRRDcmZJZVl4wSqcllZ19n6XQ3SIT4VD5CBaL7bly6wlafY41Soaugy8K3+dAZyjmuEV24+tcNxhmMwKaCNuX1dDIhSTqtEdO05iGFo9XrSFnlFdmXyB/EDRsdaiBcXgmlIzem5jWdTrcrlO8VuijmIHFEe92Wg+Mh53ieR4pPpzstK1ePLAwptg/3fDc1ZZUE5/3oDvsdp8hXqPcbZps9CfYvZ5Bnwszo/CVkzHhP/ItuzAFnimAp/f2OprVQW466oKpdX7z1PcgmL4r98nh/z7M18ll65g66GMwgT7cAHYp7D5fazcETG2yBHvftsc5do0KJQVLZGgdSsjFhaWXxiJFcbVWAjl1yEbIGng5OqeJ7KdRDpwQkNue8lxBumDr6xG4mzixLBuNbhFWv4iDH8rFcuDqSIEmNkaLb2jTFAvBaH4+MqKF3SxgqAOyNZHx4KhZd5KKUZkOy7PDQmw0qhROMz4nEFocqdA9GNButm4LZB2oqBMNt6AG87rUaFLhwINHmfYbeCthEWAg++Y5fgTZzItgis6FBTY5poQxyVe57+TY+Bya67dmz5sQgO9ERbja0WvWUOTcxgr/bhqw1510NnKEaW3yrfic+lq0wTdN6xyqq0Aqy55LGdXn54uZ56iSSPLRemqxNP6tFkoXNqxWN8oCEd8U6iVGH9uOM4lNKtK5od5XFG52jvtdjiEA2XXUyo4b5Q4FlFXeiVq1N4aDsNmcLR0YYpyU3fyq4mjz9E/7JIJmc+Q3JYnxUSUcEB1eukIfSRQ9MmRRUQ6St45SLLmU293hTcNHIUGCb4lMiYmJqazWbOPXT7zv6AzxdEXWUDlHxZiqxzuBWSjR2hWgNBMx/Sg5GVz1XoWA3iknEcNOOZ7hvjMGJoNSf2XKySMFXaqu/NjEsZbWbitPXcEyk6eXeovpEOdXi0tbwdMtECWrSAeeEFRJGvgk+IboOwueP6UgVFyYQOs0DcDXCVTGW9SVYqRAwnEGQfXj4qdTo+MDBEIR7xYTHLQfqAHJ9RvVgTxIMVTXo2Q7l6vproF/HcD/3fsG/Xn10oBcKu/ORVcmyGG6kPKJjtWtDqGJPBdH+MqDuwAq7N+f69+6odXmqIacrWzpt8PA2c3YJz/yTWBuU+bZ9qwM9Oc0gdol6R8X15KLW80VrMvozHv4nn9LxV02a5YXr3AonlfEbMwMvbXEOJFkD6ZCMyU71Z08aRWILyCCkxtZujPkkfzfRy7zYnXlsEJE0649gp10Z7r5WwrZgQQ7MfdRaOoiwDyzLdbvx+r4kkPybWG9O9KSGYSzpQ62w+2uHIn7WH81kOY2Chrx1MtkilXVJVpMXqGXu6QjISv/5ngSBSIy0OQGoTX1Ew1iF6ZmUaxhrCvhvtzaiag1BwKaEp4W+4q/1WNa2jxv4HmGP0Wq1Ixd4eyF/TaLMp/FYebTJfYU4qBBhSraOY6L25LWUgk6go+jqvXOVMrmTqeJ+1+kbgYsxkpYVDIRYwwNoZXvfqWnFo5nsWb3fv/qtPOX+C0GkisisA5PkkZTE+6IqXdTwMwKbzw70Oc4CRIIV1w2T73iw6yuxfn9KCPhi9Gl8Ma6hB2K4wACEYHaavNqdm52fhv4HSDe4vSW5XPjBx3uf27NPfJzRyepcLLU5k44jz3gjoc05UnJHbaGBGGRwVCVr9TnPzK68Ia/98/ORCHxCUOKdp1SfG/+2mWLmc1+fzakK9lnfI0VjLmUttaYN11qCy3LmS79ZNV7vy/4Uwwd/1yZtEo2qq70lbS0qHy0CP4ANQEHSyEsr6emoVu5QAoMOdSrSWaCHUZ+7vH8iZC7Ex6YJVr+zeSuLAZixBy+Jrpqcsq8K3Jv07cAWNeZYqoPVEmJkG8YbdZeY7tNIMKp5poWAELYLb0M344rAKdh3S/g2Ya1p/S9S+Eof3rde96ol/rIVyb/DGtafDL+y0dCyOrzGHI4mXM9M2zSHLL/sZdUZIcCNaZ9zBHRqrVmhc+Z2XrWA5Qi7V6UDWOwOTjp0Xy9lio5cAupzrrzAVgP7vO68G+9uhJCqkTpzdez0Wbum6A6XfByWXAnvS6jun1/WB5N4vI5lNHrwqAjuQ1aeCt/7MyozIktNhSapvdodvgdS/2MgR8ugwe6dWH5Uz/ba8C9zwCWSHiGdAoW03NQhEcmK+sl58yiF5vLd6AqthfSouQ6TA5gN6Irgo5KYebpjRAjTjf8V+xC16XL7M/pF++Hk98XssaLMaucGQfa66mv8h+ViPkYBs6Pm1EGF8jDPsk/E5rrDazpwfnsvkuwV1aykkA/s68Cnfm650Ez1e/d4WJBWT8Xhm8UOr07zcoFcC/nFSzgI+lujcfB7T3B+JkEJqBShvFgr4/WwIjUh8zpV4Ps80HJ0soj7XAZByi6JKV4IgIgiG93iXk+wP7h1LhQzq4fpBNPpGfP3cf1Eth/tbiarFkaksnW1b2vVPQaLINEDRt3BEDBYbMDlP96WLhs4GgfiPwBvaaoEckfIX8+Cm7liBqetTn23F/r5P3m6gTB6TVDDhy82cNwvBXyJBvClCGPHduwVF6+aLV6Y38hG75rJEXu6i9Lib/eX6w9zxujgGhPFgyGaxWxuLftAv4ymaQyyFqrV4Zuwvvi+C0kFHa9YxYrvAOXk4ih1ar2u+bCVZ4N8ne9vfPRghjgyvIHAKmCRc+1dqW0n+OnidH/lGylGDA/qMI8ngjaa9b17PLVvQxg0boZkDSdIrhzaAPDMzfrj60zYHwlSj74MTJ4K1smfG3wPg3nPeAxncPO5jQ71w7k5K6wIg0YAEVYSUZ2AgAARXhpZgAASUkqAAgAAAAEAA4BAgAHAgAAPgAAAJiCAgAbAAAARQIAABoBBQABAAAAYAIAABsBBQABAAAAaAIAAAAAAABFbG9uIE11c2ssIGNoaWVmIGV4ZWN1dGl2ZSBvZmZpY2VyIG9mIFRlc2xhIEluYy4sIGR1cmluZyB0aGUgNjB0aCBwcmVzaWRlbnRpYWwgaW5hdWd1cmF0aW9uIGluIHRoZSByb3R1bmRhIG9mIHRoZSBVUyBDYXBpdG9sIGluIFdhc2hpbmd0b24sIERDLCBVUywgb24gTW9uZGF5LCBKYW4uIDIwLCAyMDI1LiBEb25hbGQgVHJ1bXAncyBNb25kYXkgc3dlYXJpbmctaW4gbWFya3MganVzdCB0aGUgc2Vjb25kIHRpbWUgaW4gVVMgaGlzdG9yeSB0aGF0IGEgcHJlc2lkZW50IGxvc3QgdGhlIG9mZmljZSBhbmQgbWFuYWdlZCB0byByZXR1cm4gdG8gcG93ZXIgLSBhIGNvbWViYWNrIGNlbWVudGluZyBoaXMgcGxhY2Ugd2l0aGluIHRoZSBSZXB1YmxpY2FuIFBhcnR5IGFzIGFuIGVuZHVyaW5nLCB0cmFuc2Zvcm1hdGlvbmFsIGZpZ3VyZSByYXRoZXIgdGhhbiBhIG9uZS10ZXJtIGFiZXJyYXRpb24uIFBob3RvZ3JhcGhlcjogQ2hpcCBTb21vZGV2aWxsYS9HZXR0eSBJbWFnZXMvQmxvb21iZXJnIHZpYSBHZXR0eSBJbWFnZXM/IDIwMjUgQmxvb21iZXJnIEZpbmFuY2UgTFAsAQAAAQAAACwBAAABAAAAWE1QIJUHAABodHRwOi8vbnMuYWRvYmUuY29tL3hhcC8xLjAvADw/eHBhY2tldCBiZWdpbj0i77u/IiBpZD0iVzVNME1wQ2VoaUh6cmVTek5UY3prYzlkIj8+Cjx4OnhtcG1ldGEgeG1sbnM6eD0iYWRvYmU6bnM6bWV0YS8iPgoJPHJkZjpSREYgeG1sbnM6cmRmPSJodHRwOi8vd3d3LnczLm9yZy8xOTk5LzAyLzIyLXJkZi1zeW50YXgtbnMjIj4KCQk8cmRmOkRlc2NyaXB0aW9uIHJkZjphYm91dD0iIiB4bWxuczpwaG90b3Nob3A9Imh0dHA6Ly9ucy5hZG9iZS5jb20vcGhvdG9zaG9wLzEuMC8iIHhtbG5zOklwdGM0eG1wQ29yZT0iaHR0cDovL2lwdGMub3JnL3N0ZC9JcHRjNHhtcENvcmUvMS4wL3htbG5zLyIgICB4bWxuczpHZXR0eUltYWdlc0dJRlQ9Imh0dHA6Ly94bXAuZ2V0dHlpbWFnZXMuY29tL2dpZnQvMS4wLyIgeG1sbnM6ZGM9Imh0dHA6Ly9wdXJsLm9yZy9kYy9lbGVtZW50cy8xLjEvIiB4bWxuczpwbHVzPSJodHRwOi8vbnMudXNlcGx1cy5vcmcvbGRmL3htcC8xLjAvIiAgeG1sbnM6aXB0Y0V4dD0iaHR0cDovL2lwdGMub3JnL3N0ZC9JcHRjNHhtcEV4dC8yMDA4LTAyLTI5LyIgeG1sbnM6eG1wUmlnaHRzPSJodHRwOi8vbnMuYWRvYmUuY29tL3hhcC8xLjAvcmlnaHRzLyIgZGM6UmlnaHRzPSLCqSAyMDI1IEJsb29tYmVyZyBGaW5hbmNlIExQIiBwaG90b3Nob3A6Q3JlZGl0PSJCbG9vbWJlcmcgdmlhIEdldHR5IEltYWdlcyIgR2V0dHlJbWFnZXNHSUZUOkFzc2V0SUQ9IjIxOTQzNTgzMjMiIHhtcFJpZ2h0czpXZWJTdGF0ZW1lbnQ9Imh0dHBzOi8vd3d3LmdldHR5aW1hZ2VzLmNvbS9ldWxhP3V0bV9tZWRpdW09b3JnYW5pYyZhbXA7dXRtX3NvdXJjZT1nb29nbGUmYW1wO3V0bV9jYW1wYWlnbj1pcHRjdXJsIiBwbHVzOkRhdGFNaW5pbmc9Imh0dHA6Ly9ucy51c2VwbHVzLm9yZy9sZGYvdm9jYWIvRE1JLVBST0hJQklURUQtRVhDRVBUU0VBUkNIRU5HSU5FSU5ERVhJTkciID4KPGRjOmNyZWF0b3I+PHJkZjpTZXE+PHJkZjpsaT5CbG9vbWJlcmc8L3JkZjpsaT48L3JkZjpTZXE+PC9kYzpjcmVhdG9yPjxkYzpkZXNjcmlwdGlvbj48cmRmOkFsdD48cmRmOmxpIHhtbDpsYW5nPSJ4LWRlZmF1bHQiPkVsb24gTXVzaywgY2hpZWYgZXhlY3V0aXZlIG9mZmljZXIgb2YgVGVzbGEgSW5jLiwgZHVyaW5nIHRoZSA2MHRoIHByZXNpZGVudGlhbCBpbmF1Z3VyYXRpb24gaW4gdGhlIHJvdHVuZGEgb2YgdGhlIFVTIENhcGl0b2wgaW4gV2FzaGluZ3RvbiwgREMsIFVTLCBvbiBNb25kYXksIEphbi4gMjAsIDIwMjUuIERvbmFsZCBUcnVtcCZhcG9zO3MgTW9uZGF5IHN3ZWFyaW5nLWluIG1hcmtzIGp1c3QgdGhlIHNlY29uZCB0aW1lIGluIFVTIGhpc3RvcnkgdGhhdCBhIHByZXNpZGVudCBsb3N0IHRoZSBvZmZpY2UgYW5kIG1hbmFnZWQgdG8gcmV0dXJuIHRvIHBvd2VyIC0gYSBjb21lYmFjayBjZW1lbnRpbmcgaGlzIHBsYWNlIHdpdGhpbiB0aGUgUmVwdWJsaWNhbiBQYXJ0eSBhcyBhbiBlbmR1cmluZywgdHJhbnNmb3JtYXRpb25hbCBmaWd1cmUgcmF0aGVyIHRoYW4gYSBvbmUtdGVybSBhYmVycmF0aW9uLiBQaG90b2dyYXBoZXI6IENoaXAgU29tb2RldmlsbGEvR2V0dHkgSW1hZ2VzL0Jsb29tYmVyZyB2aWEgR2V0dHkgSW1hZ2VzPC9yZGY6bGk+PC9yZGY6QWx0PjwvZGM6ZGVzY3JpcHRpb24+CjxwbHVzOkxpY2Vuc29yPjxyZGY6U2VxPjxyZGY6bGkgcmRmOnBhcnNlVHlwZT0nUmVzb3VyY2UnPjxwbHVzOkxpY2Vuc29yVVJMPmh0dHBzOi8vd3d3LmdldHR5aW1hZ2VzLmNvbS9kZXRhaWwvMjE5NDM1ODMyMz91dG1fbWVkaXVtPW9yZ2FuaWMmYW1wO3V0bV9zb3VyY2U9Z29vZ2xlJmFtcDt1dG1fY2FtcGFpZ249aXB0Y3VybDwvcGx1czpMaWNlbnNvclVSTD48L3JkZjpsaT48L3JkZjpTZXE+PC9wbHVzOkxpY2Vuc29yPgoJCTwvcmRmOkRlc2NyaXB0aW9uPgoJPC9yZGY6UkRGPgo8L3g6eG1wbWV0YT4KPD94cGFja2V0IGVuZD0idyI/PgoA",
    "https://upload.wikimedia.org/wikipedia/commons/thumb/5/56/Donald_Trump_official_portrait.jpg/250px-Donald_Trump_official_portrait.jpg",
    "data:image/webp;base64,UklGRhgYAABXRUJQVlA4IAwYAABwjwCdASo4ATgBPo1Cm0olI6wwJtRqqgARiWVuVDU4LqShIFYQ64oMPCLsWLswYvMOZGPvDnTQ7+Vt9lM4x5bwdftpN1kDFTZrgfAJRRhWXot/huMLERPRKBu31g/7g2t0CNP/6HrYyzKqHeTYobjfXJ2vG6Zi39Lp3o6rJq8fys12XCTRY3zqI5oe+rnH/b+QAwqyE12NptvxrjwJSi2Ozvoa/FQz/Vwt/PV4Pwr/ZqEGK6QGUsrlEkwHYnTAp8WV0ocaX3EB+uSUUgqjC/QYOokKxNJcjkoH8AYoJUmsWQ+rd9fhxgeURG5HLp38XMJnS2wmxfc7Nd4BwBzV7n8ArdtuSprRgDB67AZsnVWDB50cxZXNOC02kvrSU1hNVyVF3r1hLjP4N09D4ku1BqN9ktDZV1CwYpDhX/BmClZAcvca6E3ftLEqZ3s9qPzmhJ7BcpYqtb8hxdyVMuODS5EyqksnTcFq0Pf+NEtDKHaXv7gD4U30uvWR2MoCdTPhnas2em+vq6E3ezArmjEiHtR/aO/IgdvX5nZYpV64+WKK+fBOf+bhAGWwqNyqFoPk4SJO7EGPHgB9CoRyXlmC66z5v4a8TGfL9yG5qI29Ouzv6fWSHN0HBk6dJmFLq+R2BEDBrh2ES5OdQ/Kv9JhGSeD6sQbpJ19FkKjzyWf7gXjqtHPUyH1YRd+SsU0ctPtiXDiH0GWBnlNLXhfny3eGNPhXcmf5F4V9t2wABtUcnXXngaY+ihisI8SBc/dyBy/ZGdY9xSWcntz/hFAMUfKR3F8ZpTIJbyUyv2Odsuzw2zjFYPsA+EaUXRcDts89XVtqMVxrIPQ0gzMR5MDtLY83RY6m+DAtsotAgteI9GiTHmmYRg3Mz0LQFOWnRZuW6CN8vXUtUa0ByaQhvi9lP1vjUaEsskSv+C0rMUL0rGkgMfVVJpo5D6OTWvOaDyEoFOVDdaOy0h3LhpAZqGmc7g7g/Y71WgSLVTZCWw+dp/CggVNpNo8r4bnz9eLZTfXmIQWsYKsK0nw68a49gO2PSpcR1QNGwc9MlLCBzx7IcXTVEqRN0cuVLO//QbH/7QpyHT8EgEDFjllwDXJqJxQAV+sJ5xbJrd9J7to6akgA7WHPEVRkHPhzikv7I22fBFkh8NA+aLKiExKMV8QLwikGtdNnvaTMwVcuzx1m9QuKcZoN0rtyl66GXmbGtM5sNZHUYjeVMY/90La/Tnu6bloQlyRH50mNO+7ID4XhZF9HKP5yuSUwrM53IzPdmGwbPFHoaGoLiaxkLu48njrE4HQtdcbqywOqWGdoAmoozZome122fw9LP0jUGNjNxaIzRhqXrvBbFlM4gAX/CbdjNcQc7MVCfT62n3Rj1E2r7fqhbfc+uFnVDFf7qImwfyeaM6GthoK5gR7D66k6Nb5ij/FwKtacmQ+l8Y4dbrNGnZnOPiXo/BQQSyLbaVrw5TDS6ys5Ej1InE0quglwbIzf5l4sK1DZbZTjB9ZlixcCO7uZ6QA/xL9PRPdq5sq+4ELDxn89Rmm7zGzATpvEAP7w98u6TN9cSNNd33SP1MIuxcRpg5RMY3Bml5nPZIkB7rec+nU/cpoSGWBg6MQR60Ovd7Szh4JQi5ILI/+jtJha859SG25esS6wPQbzPghaacQjsdAmAMM0yJ3AdfPBA3R7SeClGttHjRebuHHsV6gWYyYnVFmj0f7lHDqatOQ79u4MBi+znrd97OITutraPE3cimceHLAsHWMyO2bjTonFpHEdC4OitFPgYT0Ni4TCwfyM0jO7PXc6ixRg6MOeR1GCbZNGpsWYeh7Ermd4rd+f7dCChwlWFGR1d6HYSzgT8tMbLwe3XjJ3xWLsCx8Iwc8CvTqBjJBUCRzUnsfiJyaK9GwLztaqSmiHnOgCfieNUTM3op6GzUmtrQf5DgZVIviip22779CB9mSZwuq62YvPkMzbIn2e+gHxwO6QoURUI9wTHg23DnZzlkBWnXdUCcDwncvHmGB3Ba8xw3fez1zujlLg1I2NHfi2SrAOiXK84dKBsu1L8N0Yz+dcW5OTT0frz0FgBrw1Tnr6vTfAkKziXRDD4DHfGlj6jYYg7VUvb7enrgE86ts2AcKJacg5mR0iWavHZwiptcEL/LEPj8MvCNGjdml44conIkCxhkEr8qXg0EpVRc1CWD4BPIfeH3jgVjQR8OPdoCod6b3xz2uHs5A45RFwrVMewWMe0/xcHkwzZJeliBhArxam+TKAfhfFUBZNgyqVX0X2zjxemK7Uw2VMz75LZPVXoZe4TY5TdUVCtrgIEIw4PDF/cTM79V1dtqNmW5u3W2ix5HNwVyPxWl972k0t6Ayq6eSPfdTmqe/2MkucU4UrlpQopJWKPrKTa32Y169+9t0qdpUQD3e/xILmVuCA8CPJHN8gpZ6TfXQ4gx0AzYY7ifOhm5IFmY0LNTaYrzbEFqk73mNf8v3A7AWTgijsYqcvL8e1NilS5tR90nTcR+a7uEe9vZ/gpLVJi6D8NjANeEYkpkdpOdY2fH/6m7kxTIpbXrCEvyoAfGudPLQCoD5de+/pTP1mLHDTMoVgpUuYPdw27w4cFUKDeaPy8dn/dxthrpi+lpibdywKoiS1ljRj1S2RNNkkXZtQqK+e1Heg1QGGPXRvbQ5Hn6yviLv31N2fn5IxP59iCfHxTS1xT18dO1LVaxBGqIVugkKGOVeViekbUljeWe5fRXbqKZ3iYSi2LYLLHLyUDmwUg28ryZlq+IbAJ7WyLSuVYUELsu3mjPx6T7LYa+AwWFatbsJlg6cED6K9lu6GdwZg9fgyg1DC0vaNVXrcE76q3MhjVay1zdQCblZBHamS1vY6ugkLlIELgMjelSeQMK/snwjsOxWkgmv5ARu539XhdNUYsX9jpTBHRgJn9OvRPnQ0zl/n3Vf4CpfhLVy1mG8iTFNlXUsRTI3d3pbp+tWupsZKacvpuozCh8IpEvDJ0Pc/YFytcJ5eT0zRX+DpNNDWpLz/w0B4wWuaj5ma008ygKHI7A9Y4FecUFjt0zJhK4AmD+QcI0s2JeXtSfTAakp6K9xe54dYx9mDwCtcy2s1NIlxBA76lhCY2fCdQ5vods3GQMGelQQvj8y5yBf+1zxAy6acoITR4swt3QbvKBf98EP5sxnI77ZuJbAz6AVfZWYNVYfN4BZ1HGfCNMFLRDclVOfgIXc+P02t5f7nNDf9jAkWkVW8gWgzMawEe1hZbaEEXMJe5CXrfozjw8v5mVXAgo27CUGDWzphgVFix6wj12TgKuOkven8A5mGuOyrK/jRWiKADl0HKV5ah1Fz0ETGQ5nZfVyNrd83aqV9qex21d468I+IeGy/qQl4Es42LA3hq3A3tZwFGBulaCy1KK8iVpQEN8qs5BME2z7INRBb2KA0ZULXw+BHtgIM67RRyj+SezNM7aao5sRN+KetcRUr40twilDSsIOTGq0DgkeYcX0zJnHarZi52AWJPyhU4vD7hcQWgF90mjfXAbxeF8q9GDO1N3AbAt/O09XoPo+v+zUczBHPq0fQMv0y+0P5aD8NVxN9qPGDLAc51MaVTmuaaR3FYcXVV1jY2plpmID8zOws+c812ZC8dAnKjrePsPnsyfj9f7hX/l66CVTnbF5Uq8mbaWAAmvST5fgcLC3EIpAYtiVq7BW+Eki0qSZJaVeTzWkwI+/Qf6+vsQNzYp01xqgwzHguGQQHZYCuSwP3EGtXgN1lNzcWxUs2SEZVQSp7x3Zf9lhYo2M+47zIzMV5Ns7BewbTATuBrRKLtU84IwNlYgDNgxqUVck1xdbcmeDq14uJNAL6aGnY1Izixu82f87vuB118H0COvfqytdsoFPa4PFv1nkfKAiwZ3h9yj41Z2+Ai06zSR0ngfj+X0bccEghLHOM+EpIwLBBP+IJWpMo9+TjiHIrFkWVwUOHxVNEI19ZkrwJO6M0zzMbWUDrBSKSTovXYyqGn7pzBZUasKohtsxreN+/zGjBhI3Z6Ut17EP9KzHWIA9GqzjqQQh8DZUwVAHK0G29b79CyjLhirO9VCz+8/kVGMoOXDu2ymj5L7NLEiBnhkvX8H4umUFFR06V4B9aVbLQXMQsmbbm2vPDwgm911BN776KGESsP+NaPh9VlcrY2ZD2xoVedSteXKJRYYfrSck1ZCPeDnpQ2GdiWZ2iNnCgkeetDo8g8z01TBk+k/QAjlkKrHijJn3nNHXoEWHHcpqy9o0oQ3yjdkYRel4jjVkVog5VVLY5y3eIAbNoj4tHDogQDATpmiSYc720qWk6KkNp2c/nIP6/om0Ztaq5Byb3MpQL2aaWBDOyLDYN+uBDFrjqRcB7B6YA3/1jhtJmkPKjLof8NbNZUv0FjpIYHht8HkYg8KD4cOzy0uoDsnFVbSWLQ8Nyx3JyxBBZIiGaFQWwPFBuYe2EVyh5aJ9VbdrUsUjGnMg8EasFVIX6zmivAZ+n2ZLcw0BBrzxOu0XXfIVYuk3CjiZaoXIv5v0qBH4NT+OUi3ruLkgCBfno7qz2cjqbRQ6rFH3QCZNQaDoehNcd9SE3CzM9ADorn8vajC0pB9waCPxFh0dGSN5AeUY95sA396Mawta5K+8PrCbonXjX57ujHVUK+5pteKSFTpuACJCR4OvJkOShxOrziB98IfXKW+pMPxbDM/uD4mjzR6UH5IFyhOzi1uNcQhCMTGrmx5TQv/2sdB8MUZEID37Gz9im89ja5Zfg2eFSlAjlDk2GUXDPPXoRaFRURhITfy36QPmYEX2X8hWHyf+PWbKOmUNFE4sxzf2+BK6Ku7afVg33MOn9zMFkTl0ajsiVGIEsXIy6YSJ2kCixXXDVBjy1ONq9g4H/mo10SUOthBWVVYEr9PWRnno2QaeLdW5JpHWL1tSoKLIK1WoaEW4s+gTf+th+icFMlq042cQ2qnEaZWnNIc169jtfAaKeT/FdPtBf10huObSFBOxyFK6hGS5JtBtqTJTxYGRSJGRho+XXprpna/90toUqMCcEE874pVjHSIzl0Osz9LOTImD/414tUjW/RGsZ39pp3RXUfOMugRzdshjsReo4dlXp3XQHh3xLf/aIsus0gcyx9C9MgdouYa6eSnlBFih4sFGUH0DlgJNoqdpzAjgRxNjFnOviG4dAzWl9uF0hPuP91JtGks+uvifEahjFV2nR5p8SvgDCPkI0fgf14tN0HSfMDxFQtjeDuum8yXVDQcDReLv9lIFderfa+Asvqyo2/K/qCHaVVaML+PQ5C7I0dX2BctsuNL7EO7hOejAWFmAA5nrhzY7j7UmV+ZAWotcOPsrXRnBHhvSFbhteD5HMrJnackK/ZG3qDcS++KwJIGbIUmh/2xMrPh5PU0Bixz+PP5NBDUTx31WTR9EGfctS3EArTOhn+UL9VJgdGfaHjCb9TS+ai7QqHV1vNCBtPu41LX84qOS++IhqwrEy9vCcnefCdhcgXNWumnJaB7foXD25brmMCFgpG5c2y6/BrIye1B/67pf+HN443x+/oeDFN9rH5YcdzW2uTw6jDXqE70+4LIL9WFIuKzultrC4TOxN/Bp737wISqqkhAncLFUmJHgFTC1mVkaEDSdRa7U6QYaAhtQNZbMamVNRU7ueUd32t6i9BVAsTI1zMTQWyehsAyimHEW/jpozUqIaAF5dTBNJh5aMDaT3/M3izvw75A7QeoROPqKNV2YHhtgMQ6YZHFHdUdvzvWrpDctVKZgHvHH81f+IhYergglXKXFIKVzDD0MkvmS34tQJmlwE4cRRwyy2VSNShLvB+FM0qYvJ5qzqnReUF+eTfbK1Iozb8NgISGDnNn7PDsRBioSpGDpiRuRkedQ9miag574WZuu69D1sJYfPGe+O88nQh0xwC/3LIyyp0PUpD5/F95f119V9axg3N+HG0jXI8Xq8hPFHwyXAf73ypGsfELIDsetHZFAC4aav51wtbUCxfy3ptQNWMQZF+fOE8LeIAzTAxdtjJZZ/ZDcNhMBtaAAIv8uZ3+2qKHIM+Wlg0bYyRjziCpyDXxo5sIe/Mb+/2BjSQulE38cOqEdcyxGsyOa6ZNe/+RvIN7qQK13FkjPIC5ngAKl35YizTnxi02i8BR4xGzpiFy1FjVt8oqAt/zl3hFGKXzcL2ncSFkUS1ceIKY7C34EWg1IJkvxp1aNitdvtxQVbqbyCLuyg9pNJU37F+y3YE/ex5+AbYUMcMOcCZSvQWF94NVhIyQmiIVAr1Lg6WxuSBk3zrB+ZKqZeNE0FETF/rXhyzSEc+t4xunOIwmO5yAEY5Aswv29HK+bcjsSBCfYCLO3//bxMRQNpbHIoiqM0KUB43CAFWgMGHJ7LjTDeAetb39EUJW3EFvH+JSfwtcWXd4wr571G0cfJ0pWfbGMR+nt6WdxGMrARrt4NhsMgU4T+5+wZ5eiDJLeqBwVoYSMSBct+zg8wWXrm/O9c1igTMc2J5C+SvWYPsm4nj3XHXYLpDGzg6IFlvcsed46guXT92xzdAOZ0K8BrvYrocQcBiFM0B3RLKlhJmR86qjhlGsnQmI29zp3+uuvgOMWLf/DYQeH/6A9FyUYuT4bh27OyZ5oYjP7Hn8SEEFqxyHXpjYjImsTIBClBnxNXPtykemECaEDeGJK3mZAupfTqt4Oul4JxmkBX5uN3gliYYwSEn3QhcOL5iq7fpIySpDBZb9BHGxAOpO1MtzxlifSaAkjN31TkT2AAWJiKEpwEB0nJMsK6Z2Jn+T68qouqH/XUEXhsNXiO3yyLcnzIL7rKmnnGXFcUM5VZA7rsVcYpl5P4XpZb77X+FJ3SD0LvyUjJ8rim4Z48de2V/z+OUoLalA/k0KeRFvXhnsY14eixYtgo+L69tzte4xog3ZkaLD7puio7a8xRTH6NnineUmvH839Rr6bk5OezMrkyqg/n3MlBjYaLTcoZJc2kDRUbXTIoB+XX/O3U/mH26WNnSioV4DJbbwdj2AeMvIcnCDNcFjk7K3zpTPlaOSo8subqOoCzH/aVlKHacJ+LH4/W1gt+5qS0Fxpsurj/QPWwrccxGp60KEJgmZFHQCx5AfvTHroS2SincKARP3yBuJYakU+Qu5tlLIo6Wg/usdVHPvYq+Z3A6KySYvU7SxuDy0jqDyBzfTkSHDU4MEXIZ5MFV6ix4qY17v6uLAp8B1wjS60oiLCkQJbF8QHJmpXKDNfNc+LIbXt7h3q2R1pGlkKgamwEn+B/H6egyZhN2LakS43jTINnZFPaBZSma0scJBemIFfBKXuM3/uormolhkoC+VPneFuI+eeQYUZaHoEIRvKzOLdRJNvRymnERyEPNXDVTGfBJQrXWN0zvfpgk0bbr6klMRd/+g5O7+wR68E5S8MnFobWDdvMTt71RI0HRRjOtameisnikAzzoFzXEns0AA+PUy+Ozim4G5mpuGriQeN6S1tGST0dYZSGw2EhesL410zJUbAkl1zIXwpxbmiHFcyn/6qvb3pcwxaJC80NjQjtmsZvwd1FGlRCNA3+laxHh5fwB/xSuUWzcaPkiZ6oBpacFZkBp4w4IoRa/4Z76sK0BUgO73YX+lNh0Iy1+ZIfZMPGGTj0Up6gMY9uhJlrlGo9jMtrMQWOTk7Y29lkr22ifWB7E9xbzJyQit9iBq06CAlYC7VE5T9MBs26wN3/zTjlJdGiZPrc4VgXy2QkOW0F/Wkw10Bcb/mu5sYX4tabGYbXNrnK50w3NtSQH9sd6tsrnv87lc41fc9RY6BFkT3/5h+sbCFzckRru+/n7Im9xHOnK0z2WHEv/F1TZInh6rC4aMxzuhGQz4CFmgez9gAH03Na4V6fG7oX4bE9tyBkc2lhvHM5LQyj61l8/RzMgtjksgcijtA2aWsyzKe8P3C/I+XKD1S3+tCBOPMKdumEnk2d68hxBaUnmTACmhySkOhhsJa3R6Nvwux/VPlPuexaR0+yfpQgTnIvthOazg/GhFOp5+Sr3a9UYLN4Rjk3lsdIITSEarimmOr07lxjUsMmKJMhiMFd1Zh3YEnQpR2KqfnRkNbILCPXVKVFPAG9d4a21a7Hd3fojNYFpoYX8onV/qRBPo0WxgLZGAFc3bZOkwER8EBXlc//5beBJk2cz1lPt2L2Gs5qPBiLjCUPqG2TDvljp4zagyzpLi0Rcri24W4eRBn0nqoYqvPe1/G2Ii1xA6pQ7uE34U4dpD5DAhg3kw1aSy0EAaEAAAA=",
]

# 準備長條圖 線圖 總篇數 總次數等數據
list_freq_daily_line_chart = []
list_freq_news_category = []
list_sentimentInfo = []

list_total_articles=[]
list_total_frequency=[]

weeks = 12
cate = "全部"
cond = 'or'

selectedCategories = ['全部', '政治', '兩岸', '產經', '生活', '社會']
# selectedCategories = ['全部','政治', '產經', '生活', '國際', '社會', '兩岸']

for query_keywords in list_pkKeywordSet:

    # Filter news   
    df_query = filter_df_via_content(df, query_keywords, cond, cate, weeks)
    #df_query = filter_df_via_tokens(df, query_keywords, cond, cate, weeks)
    
    # Get line chart data
    line_xy_data = get_keyword_occurrence_time_series(df_query)
    list_freq_daily_line_chart.append(line_xy_data)
    
    # Get bar chart data, total articals and frequecy 
    cate_freq, cate_occurrence, total_articles, total_frequency = count_keyword(df_query, query_keywords)
    list_total_articles.append(total_articles)
    list_total_frequency.append(total_frequency)


    # We select these categories to display: ['全部', '政治', '兩岸', '產經', '生活', '社會']
    cate_freq_selected = [cate_occurrence[cate] for cate in selectedCategories]
    list_freq_news_category.append(cate_freq_selected)

    # Get sentiment information
    sentiCount, sentiPercnt = get_article_sentiment(df_query)
    #sentiInfo = '正向:{}%, 中立:{}%, 負向:{}%'.format(str(sentiPercnt['pos']), str(sentiPercnt['obj']),
    #                                            str(sentiPercnt['neg']))
    senti_info = [sentiPercnt[p] for p in ['pos','obj','neg']] # 只要取用pos,obj, neg的情緒數字篇數百分比
    list_sentimentInfo.append(senti_info)

# We need all the following data to display on our frontend page 有一大堆數據要送到前端去展示啊
pk_data =  {'list_freq_daily_line_chart': list_freq_daily_line_chart,
           'list_pkNames': list_pkNames,
           'list_colors': list_colors,
           'list_photos': list_photos,
           'list_freq_news_category': list_freq_news_category,
           'list_category': selectedCategories,
           'list_sentiInfo': list_sentimentInfo,
           'list_total_articles': list_total_articles,
           'list_total_frequency': list_total_frequency
           }

KeyError: '兩岸'

In [ ]:
pk_data

{'list_freq_daily_line_chart': [[{'x': '2022-01-29', 'y': 5},
   {'x': '2022-01-30', 'y': 5},
   {'x': '2022-01-31', 'y': 2},
   {'x': '2022-02-01', 'y': 0},
   {'x': '2022-02-02', 'y': 0},
   {'x': '2022-02-03', 'y': 0},
   {'x': '2022-02-04', 'y': 6},
   {'x': '2022-02-05', 'y': 13},
   {'x': '2022-02-06', 'y': 3},
   {'x': '2022-02-07', 'y': 11},
   {'x': '2022-02-08', 'y': 12},
   {'x': '2022-02-09', 'y': 13},
   {'x': '2022-02-10', 'y': 8},
   {'x': '2022-02-11', 'y': 10},
   {'x': '2022-02-12', 'y': 10},
   {'x': '2022-02-13', 'y': 3},
   {'x': '2022-02-14', 'y': 10},
   {'x': '2022-02-15', 'y': 15},
   {'x': '2022-02-16', 'y': 13},
   {'x': '2022-02-17', 'y': 13},
   {'x': '2022-02-18', 'y': 14},
   {'x': '2022-02-19', 'y': 10},
   {'x': '2022-02-20', 'y': 2},
   {'x': '2022-02-21', 'y': 11},
   {'x': '2022-02-22', 'y': 11},
   {'x': '2022-02-23', 'y': 10},
   {'x': '2022-02-24', 'y': 12},
   {'x': '2022-02-25', 'y': 6},
   {'x': '2022-02-26', 'y': 4},
   {'x': '2022-02-27', 'y'

# Step 5: Save PK data to csv file

In [ ]:
df_data_pk = pd.DataFrame(list(pk_data.items()),columns=['name','value'])

In [ ]:
df_data_pk

,name,value
0,list_freq_daily_line_chart,"[[{'x': '2022-01-29', 'y': 5}, {'x': '2022-01-..."
1,list_pkNames,"[陳時中, 黃珊珊, 蔣萬安]"
2,list_colors,"[rgba(0,128,0,0.2), rgba(0,255,255,0.2), rgba(..."
3,list_photos,[https://upload.wikimedia.org/wikipedia/common...
4,list_freq_news_category,"[[743, 110, 1, 8, 578, 1], [133, 44, 0, 2, 80,..."
5,list_category,"[全部, 政治, 兩岸, 產經, 生活, 社會]"
6,list_sentiInfo,"[[50, 20, 31], [49, 26, 26], [50, 20, 30]]"
7,list_total_articles,"[743, 133, 111]"
8,list_total_frequency,"[2628, 384, 453]"


In [ ]:
## 存成csv格式檔案
df_data_pk.to_csv('pk_person.csv',sep=',', index=None)

### Alternative way: using zip

In [ ]:
k=list(pk_data.keys())
v=list(pk_data.values())

In [ ]:
#list(zip(k,v))

In [ ]:
df_data_pk = pd.DataFrame(list(zip(k,v)),columns=['name','value'])
df_data_pk

,name,value
0,list_freq_daily_line_chart,"[[{'x': '2022-01-29', 'y': 5}, {'x': '2022-01-..."
1,list_pkNames,"[陳時中, 黃珊珊, 蔣萬安]"
2,list_colors,"[rgba(0,128,0,0.2), rgba(0,255,255,0.2), rgba(..."
3,list_photos,[https://upload.wikimedia.org/wikipedia/common...
4,list_freq_news_category,"[[743, 110, 1, 8, 578, 1], [133, 44, 0, 2, 80,..."
5,list_category,"[全部, 政治, 兩岸, 產經, 生活, 社會]"
6,list_sentiInfo,"[[49, 19, 30], [48, 25, 25], [50, 19, 29]]"
7,list_total_articles,"[743, 133, 111]"
8,list_total_frequency,"[2628, 384, 453]"


## Read csv file and convert to dict format

In [ ]:
df_data_pk = pd.read_csv('pk_person.csv')

In [ ]:
df_data_pk 

,name,value
0,list_freq_daily_line_chart,"[[{'x': '2022-01-29', 'y': 5}, {'x': '2022-01-..."
1,list_pkNames,"['陳時中', '黃珊珊', '蔣萬安']"
2,list_colors,"['rgba(0,128,0,0.2)', 'rgba(0,255,255,0.2)', '..."
3,list_photos,['https://upload.wikimedia.org/wikipedia/commo...
4,list_freq_news_category,"[[743, 110, 1, 8, 578, 1], [133, 44, 0, 2, 80,..."
5,list_category,"['全部', '政治', '兩岸', '產經', '生活', '社會']"
6,list_sentiInfo,"[[50, 20, 31], [49, 26, 26], [50, 20, 30]]"
7,list_total_articles,"[743, 133, 111]"
8,list_total_frequency,"[2628, 384, 453]"


In [ ]:
df_data_pk.values


array([['list_freq_daily_line_chart',
        "[[{'x': '2022-01-29', 'y': 5}, {'x': '2022-01-30', 'y': 5}, {'x': '2022-01-31', 'y': 2}, {'x': '2022-02-01', 'y': 0}, {'x': '2022-02-02', 'y': 0}, {'x': '2022-02-03', 'y': 0}, {'x': '2022-02-04', 'y': 6}, {'x': '2022-02-05', 'y': 13}, {'x': '2022-02-06', 'y': 3}, {'x': '2022-02-07', 'y': 11}, {'x': '2022-02-08', 'y': 12}, {'x': '2022-02-09', 'y': 13}, {'x': '2022-02-10', 'y': 8}, {'x': '2022-02-11', 'y': 10}, {'x': '2022-02-12', 'y': 10}, {'x': '2022-02-13', 'y': 3}, {'x': '2022-02-14', 'y': 10}, {'x': '2022-02-15', 'y': 15}, {'x': '2022-02-16', 'y': 13}, {'x': '2022-02-17', 'y': 13}, {'x': '2022-02-18', 'y': 14}, {'x': '2022-02-19', 'y': 10}, {'x': '2022-02-20', 'y': 2}, {'x': '2022-02-21', 'y': 11}, {'x': '2022-02-22', 'y': 11}, {'x': '2022-02-23', 'y': 10}, {'x': '2022-02-24', 'y': 12}, {'x': '2022-02-25', 'y': 6}, {'x': '2022-02-26', 'y': 4}, {'x': '2022-02-27', 'y': 1}, {'x': '2022-02-28', 'y': 2}, {'x': '2022-03-01', 'y': 2}, {'x': '

In [ ]:
# Convert to dictionary format
data = dict(list(df_data_pk.values))
data

{'list_freq_daily_line_chart': "[[{'x': '2022-01-29', 'y': 5}, {'x': '2022-01-30', 'y': 5}, {'x': '2022-01-31', 'y': 2}, {'x': '2022-02-01', 'y': 0}, {'x': '2022-02-02', 'y': 0}, {'x': '2022-02-03', 'y': 0}, {'x': '2022-02-04', 'y': 6}, {'x': '2022-02-05', 'y': 13}, {'x': '2022-02-06', 'y': 3}, {'x': '2022-02-07', 'y': 11}, {'x': '2022-02-08', 'y': 12}, {'x': '2022-02-09', 'y': 13}, {'x': '2022-02-10', 'y': 8}, {'x': '2022-02-11', 'y': 10}, {'x': '2022-02-12', 'y': 10}, {'x': '2022-02-13', 'y': 3}, {'x': '2022-02-14', 'y': 10}, {'x': '2022-02-15', 'y': 15}, {'x': '2022-02-16', 'y': 13}, {'x': '2022-02-17', 'y': 13}, {'x': '2022-02-18', 'y': 14}, {'x': '2022-02-19', 'y': 10}, {'x': '2022-02-20', 'y': 2}, {'x': '2022-02-21', 'y': 11}, {'x': '2022-02-22', 'y': 11}, {'x': '2022-02-23', 'y': 10}, {'x': '2022-02-24', 'y': 12}, {'x': '2022-02-25', 'y': 6}, {'x': '2022-02-26', 'y': 4}, {'x': '2022-02-27', 'y': 1}, {'x': '2022-02-28', 'y': 2}, {'x': '2022-03-01', 'y': 2}, {'x': '2022-03-02', 'y

## Convert to dictionary format (for reference)

In [ ]:
data={}
for k, v in df_data_pk.values:
    #print(eval(v))
    data[k]=eval(v)

In [ ]:
# Another approach to get the sampe result
data={}
for k,v in zip(df_data_pk.name, df_data_pk.value):
    data[k]=eval(v)
    

In [ ]:
data

{'list_freq_daily_line_chart': [[{'x': '2022-01-29', 'y': 5},
   {'x': '2022-01-30', 'y': 5},
   {'x': '2022-01-31', 'y': 2},
   {'x': '2022-02-01', 'y': 0},
   {'x': '2022-02-02', 'y': 0},
   {'x': '2022-02-03', 'y': 0},
   {'x': '2022-02-04', 'y': 6},
   {'x': '2022-02-05', 'y': 13},
   {'x': '2022-02-06', 'y': 3},
   {'x': '2022-02-07', 'y': 11},
   {'x': '2022-02-08', 'y': 12},
   {'x': '2022-02-09', 'y': 13},
   {'x': '2022-02-10', 'y': 8},
   {'x': '2022-02-11', 'y': 10},
   {'x': '2022-02-12', 'y': 10},
   {'x': '2022-02-13', 'y': 3},
   {'x': '2022-02-14', 'y': 10},
   {'x': '2022-02-15', 'y': 15},
   {'x': '2022-02-16', 'y': 13},
   {'x': '2022-02-17', 'y': 13},
   {'x': '2022-02-18', 'y': 14},
   {'x': '2022-02-19', 'y': 10},
   {'x': '2022-02-20', 'y': 2},
   {'x': '2022-02-21', 'y': 11},
   {'x': '2022-02-22', 'y': 11},
   {'x': '2022-02-23', 'y': 10},
   {'x': '2022-02-24', 'y': 12},
   {'x': '2022-02-25', 'y': 6},
   {'x': '2022-02-26', 'y': 4},
   {'x': '2022-02-27', 'y'

# views.py in Django

In [ ]:
from django.http import JsonResponse
import pandas as pd
from django.shortcuts import render
from django.views.decorators.csrf import csrf_exempt

def load_data_pk():
    # Read data from csv file
    df_data_pk = pd.read_csv('app_taipei_mayor/dataset/pk_person.csv')
    
    global data
    #data={}
    # for k,v in zip(df_data_pk.name, df_data_pk.value):
    #     data[k]=eval(v)
    
    # Convert to dictionary format
    data = dict(list(df_data_pk.values)) # Just one line to convert them to dictionary


    # 沒用到的變數刪除之
    del df_data_pk

# load pk data
load_data_pk()

def home(request):
    return render(request,'app_taipei_mayor/home.html')

# csrf_exempt is used for POST
# 單獨指定這一支程式忽略csrf驗證
@csrf_exempt
def api_get_taipei_mayor_data(request):
    return JsonResponse(data)

print('Load app taipei mayor leaderboard...')


#  For reference: alternative aprroach to process response data with json format


In [ ]:
# 個別包含完整的資料
list_pkNames = ['馬斯克', '川普', '蔣萬安']
list_pkKeywordSet = [['馬斯克'], ['川普'], ['蔣萬安']]

# pkName = ['柯文哲', '蔡英文', '韓國瑜']
# pkset = [['柯文哲','柯p','柯P'], ['蔡英文'], ['韓國瑜']]

#list_colors = ['green', 'red', 'blue']
list_colors = ["rgba(0,128,0,0.2)",'rgba(0,255,255,0.2)','rgba(0,0,255,0.2)']

list_photos = [
    "https://upload.wikimedia.org/wikipedia/commons/c/c0/%E8%A1%9B%E7%94%9F%E7%A6%8F%E5%88%A9%E9%83%A8%E9%83%A8%E9%95%B7%E9%99%B3%E6%99%82%E4%B8%AD.jpg",
    "https://councilorwatch.tw/images/councillors/female/230.jpg",
    "https://upload.wikimedia.org/wikipedia/commons/9/93/%E8%94%A3%E8%90%AC%E5%AE%89%E5%A7%94%E5%93%A1.jpg"
]


def get_pk_data(name, query_keywords, photo, color):

    df_query = filter_df_via_content(df, query_keywords, cond, cate, weeks)

    line_xy_data = get_keyword_occurrence_time_series(df_query)

    cate_freq, cate_occurrence, total_articles, total_frequency = count_keyword(
        df_query, query_keywords)
    selectedCategories = ['全部', '政治', '兩岸', '產經', '生活', '社會']
    # selectedCategories = ['政治', '產經', '生活', '國際', '社會', '兩岸','全部']
    # We select these categories to display: ['全部', '政治', '兩岸', '產經', '生活', '社會']
    freq_selected_categories = [cate_occurrence[k] for k in selectedCategories]

    # sentiment information
    sentiCount, sentiPercnt = get_article_sentiment(df_query)
    # sentiInfo = '正向:{}%, 中立:{}%, 負向:{}%'.format(str(sentiPercnt['pos']), str(sentiPercnt['obj']),
    #                                            str(sentiPercnt['neg']))
    sentiment = [sentiPercnt[p] for p in ['pos', 'obj', 'neg']]

    pk_data = {'line_xy_data': line_xy_data,
               'name': name,
               'color': color,
               'photo': photo,
               'freq_news_category': freq_selected_categories,
               'category': selectedCategories,
               'sentiment': sentiment,
               'total_articles': total_articles,
               'total_frequency': total_frequency,
               }
    return pk_data

pk_data_dict = {}
for i in range(len(list_pkNames)):
    name = list_pkNames[i]
    query_keywords = list_pkKeywordSet[i]
    photo = list_photos[i]
    color = list_colors[i]
    pk_data_dict[name] = get_pk_data(name, query_keywords, photo, color)



In [ ]:
pk_data_dict

{'陳時中': {'line_xy_data': [{'x': '2022-01-29', 'y': 5},
   {'x': '2022-01-30', 'y': 5},
   {'x': '2022-01-31', 'y': 2},
   {'x': '2022-02-01', 'y': 0},
   {'x': '2022-02-02', 'y': 0},
   {'x': '2022-02-03', 'y': 0},
   {'x': '2022-02-04', 'y': 6},
   {'x': '2022-02-05', 'y': 13},
   {'x': '2022-02-06', 'y': 3},
   {'x': '2022-02-07', 'y': 11},
   {'x': '2022-02-08', 'y': 12},
   {'x': '2022-02-09', 'y': 13},
   {'x': '2022-02-10', 'y': 8},
   {'x': '2022-02-11', 'y': 10},
   {'x': '2022-02-12', 'y': 10},
   {'x': '2022-02-13', 'y': 3},
   {'x': '2022-02-14', 'y': 10},
   {'x': '2022-02-15', 'y': 15},
   {'x': '2022-02-16', 'y': 13},
   {'x': '2022-02-17', 'y': 13},
   {'x': '2022-02-18', 'y': 14},
   {'x': '2022-02-19', 'y': 10},
   {'x': '2022-02-20', 'y': 2},
   {'x': '2022-02-21', 'y': 11},
   {'x': '2022-02-22', 'y': 11},
   {'x': '2022-02-23', 'y': 10},
   {'x': '2022-02-24', 'y': 12},
   {'x': '2022-02-25', 'y': 6},
   {'x': '2022-02-26', 'y': 4},
   {'x': '2022-02-27', 'y': 1},
 

### Save file using json format

In [ ]:

import json
# Serialize data into file:
json.dump( pk_data_dict, open( "pk_person.json", 'w' ) )

# Read data from file:
data = json.load( open( "pk_person.json" ) )
data

{'陳時中': {'line_xy_data': [{'x': '2022-01-29', 'y': 5},
   {'x': '2022-01-30', 'y': 5},
   {'x': '2022-01-31', 'y': 2},
   {'x': '2022-02-01', 'y': 0},
   {'x': '2022-02-02', 'y': 0},
   {'x': '2022-02-03', 'y': 0},
   {'x': '2022-02-04', 'y': 6},
   {'x': '2022-02-05', 'y': 13},
   {'x': '2022-02-06', 'y': 3},
   {'x': '2022-02-07', 'y': 11},
   {'x': '2022-02-08', 'y': 12},
   {'x': '2022-02-09', 'y': 13},
   {'x': '2022-02-10', 'y': 8},
   {'x': '2022-02-11', 'y': 10},
   {'x': '2022-02-12', 'y': 10},
   {'x': '2022-02-13', 'y': 3},
   {'x': '2022-02-14', 'y': 10},
   {'x': '2022-02-15', 'y': 15},
   {'x': '2022-02-16', 'y': 13},
   {'x': '2022-02-17', 'y': 13},
   {'x': '2022-02-18', 'y': 14},
   {'x': '2022-02-19', 'y': 10},
   {'x': '2022-02-20', 'y': 2},
   {'x': '2022-02-21', 'y': 11},
   {'x': '2022-02-22', 'y': 11},
   {'x': '2022-02-23', 'y': 10},
   {'x': '2022-02-24', 'y': 12},
   {'x': '2022-02-25', 'y': 6},
   {'x': '2022-02-26', 'y': 4},
   {'x': '2022-02-27', 'y': 1},
 